###Client ID and Client Secret

In [ ]:
client_id = "???"
client_secret = "???"

###Libraries and Settings

In [ ]:
import requests
import base64
import json
import os
import pandas as pd
from google.colab import drive
from itertools import islice
from ast import literal_eval
import time 
drive.mount("/content/drive")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Mounted at /content/drive


###Defining the Functions:

In [ ]:
def initialize_token():

    '''
    This function is used to initialize an access token for making authorized API calls to the Striven API.
    The function first encodes the client ID and client secret using base64 encoding to create an authorization header for client authentication. It then creates a dictionary containing the grant type and client ID for making the request to the API to obtain the access token.
    After making the request, the function checks if the response was successful using the ok attribute of the token_response object. If the response was successful, it extracts the access token and refresh token from the JSON response and returns them. If the response was not successful, the function returns None for both the access token and refresh token.
    It is important to note that the access token is valid only for the duration indicated by expires_in (in seconds). The refresh token can be used to obtain a new access token before the current one expires. The access token obtained from this function should be stored somewhere global so that it can be used to make API calls until it expires.
    '''
    # ClientID and ClientSecret must be obtained from API Settings page in Striven
    # These keys are used to request a accesstoken which will be used to make authorized api calls. Storing these information safely is highly recommended 
    #client_id = 
    #client_secret = 
    
    # Create authorization header for client authentication
    authorization_header = base64.b64encode(bytes(client_id + ":" + client_secret, 'utf-8')).decode('ascii')
    
    # Use client_credentials granttype with ClientID to get the accesstoken
    grant_client_credential = {
        "grant_type": "client_credentials",
        "ClientId": client_id
    }
    
    # Requesting token
    token_response = requests.post("https://api.striven.com/accesstoken", data=grant_client_credential, headers={
        "Authorization": f"Basic {authorization_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    })

    if token_response.ok:
        # The response contains following information
        # access_token
        # refresh_token
        # expires_in
        # IMPORTANT!!! access_token is valid for the duration indicated by expires_in(in seconds).
        # refresh_token can be used to refresh access token before it expires
        token_response_de = token_response.json()
        
        # this accesstoken should be stored somewhere global so that it can be used to make api calls until it expires.
        access_token = token_response_de["access_token"]
        refresh_token = token_response_de["refresh_token"]
        
        return access_token, refresh_token
    else:
        # error will be returned by token_response.text
        return None, None


def refresh_access_token():

    '''
    This function is used to refresh the access token for making authorized API calls to the Striven API when the previous access token has expired.
    The function creates a new requests.Session() object, sets the Accept header to "application/json", and creates an authorization header using base64 encoding for client authentication. It then creates a dictionary containing the grant type and refresh token for making the request to the API to obtain a new access token.
    After making the request, the function checks if the response was successful using the status_code attribute of the new_token_response object. If the response was successful, it extracts the access token and refresh token from the JSON response and updates the global variables for access_token and refresh_token.
    It's worth noting that the refresh_token value is currently hardcoded to "refresh_token". In reality, this value should be the actual refresh token obtained from the previous access token request.
    Also, it's important to have some error handling in case the request for a new access token fails. Currently, the function doesn't handle any errors and just continues executing even if the request fails.
    '''
    # ClientID and ClientSecret must be obtained from API Settings page in Striven
    #client_id = 
    #client_secret = 
    client = requests.Session()
    # add an accept header for JSON format
    client.headers.update({"Accept": "application/json"})
    # create authorization header for client authentication
    authorization_header = base64.b64encode(bytes(f"{client_id}:{client_secret}", "utf-8")).decode("utf-8")
    client.headers.update({"Authorization": f"Basic {authorization_header}"})
    # Use refresh_token grant type with ClientID to get the access token
    grant_client_credential = {
        "grant_type": "refresh_token",
        "refresh_token": "refresh_token",
    }
    # Requesting token
    new_token_response = client.post("https://api.striven.com/accesstoken", data=grant_client_credential)
    if new_token_response.status_code == requests.codes.ok:
        token_response = new_token_response.json()
        # update the tokens
        global access_token
        global refresh_token
        access_token = token_response["access_token"]
        refresh_token = token_response["refresh_token"]


def get_details(id, access_token, api_url):

    """
    Given an id, access_token, and api_url, this function sends a GET request to the api_url
    with the id and returns the response data in a pandas DataFrame.
    The function first creates a list details to store the response data for the resource. It then sets the headers for the API request to include the Content-Type and Authorization with the provided access_token.
    The id parameter is then appended to the api_url to create the complete API endpoint URL for the resource. The function then makes a GET request to the API using the requests.get() method and passes in the endpoint URL and headers.
    If the response is successful (with a HTTP status code of 200), the function extracts the response data as a JSON object using the .json() method and appends it to the details list. If the response is not successful, the function raises an exception using the .raise_for_status() method to alert the caller of the error.
    Finally, the function returns the response data as a Pandas DataFrame using the pd.DataFrame() method.
    It's worth noting that this function is specifically designed to work with a single resource and will only return details for that resource. If you need to retrieve details for multiple resources, you will need to modify the function to handle multiple IDs and concatenate the resulting data.
    """
    details = []
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}
    url = api_url + str(id)  # Convert id to a string 
    response = requests.get(url, headers=headers)

    if response.ok:
        data = response.json()
        details.append(data)
    else:
        response.raise_for_status()

    return pd.DataFrame(details)


def update_details(access_token, api_url, path_table, path_detail, x=None):

    """
    This function is used to update the details for a set of resource IDs in the Striven API by making API calls using the provided access_token, api_url, path_table, and path_detail parameters.
    The function first reads in the resource IDs to be updated from a CSV file at path_table using the pd.read_csv() method. If a CSV file for the resource details already exists at path_detail, it is also read in using pd.read_csv(), otherwise an empty DataFrame is created.
    The function then computes the set difference between the ids in the table DataFrame and the ids in the details DataFrame to determine which IDs need to be updated. If there are any IDs to update, the function iterates through them and for each ID, it calls the get_details() function to retrieve the updated details from the API using the provided access_token and api_url. The updated details are then appended to the details DataFrame using the append() method.
    Finally, the updated details DataFrame is saved to the CSV file at path_detail using the to_csv() method and returned from the function.
    It's worth noting that the function is designed to handle cases where only a subset of the IDs need to be updated by using the optional x parameter to iterate over only the first x IDs in the ids set. Additionally, if the ids set is empty, the function raises an exception indicating that the resource details are up to date.
    """
    table = pd.read_csv(path_table)

    if os.path.isfile(path_detail):
        details = pd.read_csv(path_detail)
    else:
        details = pd.DataFrame()
        
    # Get the set of ids that need to be updated
    ids = set(table["id"]) - set(details["id"])
    if bool(ids):
        for id in islice(ids, x):
            # Send a GET request to the API to get the details
            data = get_details(id, access_token, api_url)
            # Append the details to the existing DataFrame
            details = details.append(data, ignore_index=True)
    else:
        raise Exception("Table is up to date")

    # Save the updated details to the CSV file
    details.to_csv(path_detail, index=False)

    return details


def search_resource(access_token, resource_type, **kwargs):
    
    '''
    This function search_resource() makes a POST request to an API endpoint with the search parameters for the specified resource_type, using the provided access_token for authorization. It takes in the access_token and resource_type as required parameters, and additional search parameters can be passed as keyword arguments **kwargs.
    The function converts the search parameters to a JSON string and sets the headers for the API call including the authorization token. Then it sends the POST request to the API endpoint using the requests module in Python. If the response status code is 200, it deserializes the response JSON string into a Python object and returns it. Otherwise, it returns the response text directly if the status code is not 200.
    '''
    # build the search parameters for the API endpoint
    search_params = kwargs
    
    # convert the search parameters to a JSON string
    search_params_json = json.dumps(search_params)
    
    # set the headers for the API call
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}
    
    # make a POST request to the API endpoint with the headers and data
    response = requests.post(f"https://api.striven.com/v1/{resource_type}/search", headers=headers, data=search_params_json)
    
    # check the response status code
    if response.status_code == 200:
        # deserialize the response JSON string into a Python object
        response_obj = json.loads(response.text)
        # return the Python object
        return response_obj
    else:
        # return the response text directly if the status code is not 200
        return response.text


def get_all_resources(access_token, resource_type):

    '''
    This function get_all_resources() is designed to retrieve all resources of a specified resource_type from an API endpoint using pagination. The function takes in an access_token and a resource_type as required parameters.
    The function first sets the page size and page index, and initializes an empty list called resources to store the retrieved resources. Then it enters a loop that iterates over all pages of resources until there are no more resources left to retrieve.
    Inside the loop, the function calls another function search_resource() (presumably defined elsewhere in the code) to retrieve a page of resources with the specified page size and index. It checks if there are no more resources to retrieve by checking if the response JSON contains a data field with a non-empty list. If the data field is empty, the loop is broken. 
    If there are more resources to retrieve, it appends the retrieved resources to the resources list, increments the page index for the next iteration, and continues to retrieve the next page of resources.
    Once all resources have been retrieved, the function creates a pandas DataFrame from the list of resources and returns it. The returned DataFrame contains all of the resources for the specified resource_type.
    '''
    # set the page size and page index
    page_size = 1000
    page_index = 0
    
    # initialize an empty list to store the resources
    resources = []
    
    # loop through all pages of resources
    while True:
        # get the resources with the specified page size and index
        response = search_resource(access_token, resource_type, pageIndex=page_index, pageSize=page_size, sortOrder=1)

        # check if there are no more resources to retrieve
        if not response.get('data'):
            break

        # add the resources to the list
        resources.extend(response['data'])

        # increment the page index for the next iteration
        page_index += 1

    # create a pandas DataFrame from the list of resources
    df = pd.DataFrame(resources)

    # return the DataFrame
    return df


def get_resources(access_token, resource_type):

    """
    The function get_resources() is designed to retrieve resources of a specified resource_type from an API endpoint and return them as a Pandas DataFrame. The function takes in an access_token and a resource_type as required parameters.
    The function first sets the headers for the API call using the access_token parameter, and then makes a GET request to the API endpoint for the specified resource_type. It checks the response status code, and if the code is 200, it deserializes the response JSON string into a Python object, extracts the resources data from the object, converts the resources to a Pandas DataFrame and returns the DataFrame.
    If the response status code is not 200, the function returns the response text directly.
    Overall, this function is useful for retrieving resources of a specific type from an API endpoint and analyzing the data in a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the resources API endpoint with the headers.
    response = requests.get(f"https://api.striven.com/v1/{resource_type}", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the resources from the response object.
        resources = response_obj['data']

        # Convert the resources to a Pandas DataFrame.
        df = pd.DataFrame(resources)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

def get_resources_2(access_token, resource_type):

    """
    The function get_resources() is designed to retrieve resources of a specified resource_type from an API endpoint and return them as a Pandas DataFrame. The function takes in an access_token and a resource_type as required parameters.
    The function first sets the headers for the API call using the access_token parameter, and then makes a GET request to the API endpoint for the specified resource_type. It checks the response status code, and if the code is 200, it deserializes the response JSON string into a Python object, extracts the resources data from the object, converts the resources to a Pandas DataFrame and returns the DataFrame.
    If the response status code is not 200, the function returns the response text directly.
    Overall, this function is useful for retrieving resources of a specific type from an API endpoint and analyzing the data in a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the resources API endpoint with the headers.
    response = requests.post(f"https://api.striven.com/v1/{resource_type}/search", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the resources from the response object.
        resources = response_obj['data']

        # Convert the resources to a Pandas DataFrame.
        df = pd.DataFrame(resources)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text


def try_literal_eval(s):
    try:
        return literal_eval(s)
    except (ValueError, SyntaxError):
        return s


def add_clean_col(df, col, nested=False):
    if not nested:
        for key in df[df[col].notnull()][col].iloc[0].keys():
            df[col + "_" + str(key)] = df[col].apply(lambda x: x[key] if isinstance(x, dict) and x.get(key) is not None else x)
    else:
        for key in df[df[col].notnull()][col].iloc[0][0].keys():
            df[col + "_" + str(key)] = df[col].apply(lambda x: x[0][key])
    df = df.drop(col, axis=1)
    return df


# Define function to clean data
def clean_data(data, single_cols, nested_cols, drop_cols, date_cols):
    data_clean = pd.read_csv(data)
    data_clean = data_clean.applymap(try_literal_eval)

    for col in nested_cols:
        data_clean = add_clean_col(data_clean, col, nested=True)

    for col in single_cols:
        data_clean = add_clean_col(data_clean, col)

    for col in date_cols:
        data_clean[col] = pd.to_datetime(data_clean[col]).dt.date

    data_clean.drop(columns=drop_cols, inplace=True)
    data_clean.to_csv(data.split(".")[0] + "_clean.csv", index=False)
    
    return(data_clean)

### Useful

In [ ]:
def drop_when_low_missing_values(df, confidence_interval=0.95):

    # Count the number of missing values in each column. Print the number of missing values in each column
    #missing_counts = df.isna().sum()
    #print(missing_counts)

    # Calculate the threshold based on the confidence interval
    threshold = len(df) * (1 - confidence_interval)

    # Filter out columns with missing values above the threshold
    cols_to_clean = df.columns[df.isna().sum() <= threshold]

    # clean columns with missing values above the threshold
    df.dropna(subset = cols_to_clean, inplace=True)

    # Print the number of missing values in each column after cleaning columns
    #print(df.isna().sum())

    return df


def replace_missing_with_median(df, group_col, col_with_missing):
    """
    Replace missing values in a column with the median value of that column for the corresponding group in another column.

    Args:
        df (pandas DataFrame): The input DataFrame
        group_col (str): The name of the column to group by
        col_with_missing (str): The name of the column with missing values

    Returns:
        pandas DataFrame: The input DataFrame with missing values replaced by the median of the corresponding group
    """

    # Calculate median of the column with missing values for each group
    group_median = df.groupby(group_col)[col_with_missing].median()

    # Print the median of the column with missing values for each group
    print(group_median)

    # Convert the median values to a dictionary
    median_dict = group_median.to_dict()

    # Replace missing values with the median of the corresponding group
    df[col_with_missing] = df[col_with_missing].fillna(df[group_col].map(median_dict))

    # Print the resulting DataFrame
    print(df)
    
    return df

def print_unique_values_for_object_columns(df):
    """
    Print the number of unique values for each column in a DataFrame that contains object (non-numeric) data types.

    Args:
        df (pandas DataFrame): The input DataFrame

    Returns:
        None
    """

    # Filter the DataFrame for object columns
    non_numeric = df.select_dtypes("object")

    # Loop through columns
    for col in non_numeric.columns:

        # Print the number of unique values
        print(f"Number of unique values in {col} column: ", non_numeric[col].nunique())


###Acquiring Access Token

In [ ]:
access_token, refresh_token = initialize_token()

###Collecting and Cleaning the invoices Table

In [ ]:
# call the function with an access token to retrieve all invoices
invoices = get_all_resources(access_token, 'invoices')
invoices.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/invoices.csv", index=False)
invoices.head()

,id,txnNumber,customer,dueDate,invoiceTotal,openBalance,memo,dateCreated,lastUpdatedDate,currency
0,8566,1907475,"{'id': 3511, 'name': 'USA Crane & Hoist Servic...",2023-05-18T00:00:00,155.81,0.00,SHIPPED 5/18/23; TRK#1Z2X28070370850774,2023-05-18T16:33:44.283,2023-05-18T16:33:44.283,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,8565,1907474,"{'id': 879, 'name': 'M. TEIXEIRA SOAPSTONE'}",2023-06-17T00:00:00,8599.31,8599.31,UPGRADE CRANE SYSTEM - COMPLETED 5/18/23,2023-05-18T16:05:54.257,2023-05-18T16:05:54.257,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,8564,1907473,"{'id': 928, 'name': 'Mid Atlantic Mechanical'}",2023-05-18T00:00:00,553.71,0.00,SHIPPED 5/17/23; TRK#1Z4894510329478764,2023-05-18T08:25:56.803,2023-05-18T08:25:56.803,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,8563,1907472,"{'id': 3483, 'name': 'CR Meyer'}",2023-05-17T00:00:00,17617.50,17617.50,MILESTONE INVOICE OF ORDER #200939 - 25% WITH ...,2023-05-17T12:21:10.937,2023-05-17T12:24:43.207,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,8562,1907471,"{'id': 2319, 'name': 'SODEXO INC'}",2023-06-11T00:00:00,1600.00,1600.00,ANNUAL OSHA INSPECTION & PM (MHE) - RAHWAY - W...,2023-05-16T11:35:50.343,2023-05-16T11:35:50.343,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
invoices_single_cols = ["customer", "currency"]
invoices_nested_cols = []
invoices_drop_cols = []
invoices_date_cols = ["dueDate", "dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
invoices_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/invoices.csv", 
           invoices_single_cols, invoices_nested_cols, invoices_drop_cols, invoices_date_cols)

invoices_clean.head()

,id,txnNumber,dueDate,invoiceTotal,openBalance,memo,dateCreated,lastUpdatedDate,customer_id,customer_name,currency_currencyISOCode,currency_exchangeRate
0,8566,1907475,2023-05-18,155.81,0.00,SHIPPED 5/18/23; TRK#1Z2X28070370850774,2023-05-18,2023-05-18,3511,USA Crane & Hoist Services,USD,1.0
1,8565,1907474,2023-06-17,8599.31,8599.31,UPGRADE CRANE SYSTEM - COMPLETED 5/18/23,2023-05-18,2023-05-18,879,M. TEIXEIRA SOAPSTONE,USD,1.0
2,8564,1907473,2023-05-18,553.71,0.00,SHIPPED 5/17/23; TRK#1Z4894510329478764,2023-05-18,2023-05-18,928,Mid Atlantic Mechanical,USD,1.0
3,8563,1907472,2023-05-17,17617.50,17617.50,MILESTONE INVOICE OF ORDER #200939 - 25% WITH ...,2023-05-17,2023-05-17,3483,CR Meyer,USD,1.0
4,8562,1907471,2023-06-11,1600.00,1600.00,ANNUAL OSHA INSPECTION & PM (MHE) - RAHWAY - W...,2023-05-16,2023-05-16,2319,SODEXO INC,USD,1.0


###Collecting and Cleaning the bills Table

In [ ]:
# call the function with an access token to retrieve all bills
bills = get_all_resources(access_token, 'bills')
bills.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/bills.csv", index=False)
bills.head()

,id,number,dueDate,vendor,status,memo,totalAmount,openBalance,dateCreated,lastUpdatedDate,currency
0,11402,11257,2023-06-16T00:00:00,"{'id': 192, 'name': 'STREET CRANE COMPANY LTD'}","{'id': 125, 'name': 'To Be Paid'}",I76630,415.45,415.45,2023-05-18T08:23:14.847,2023-05-18T08:23:14.847,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,11401,11256,2023-05-23T00:00:00,"{'id': 213, 'name': 'UPS - UNITED PARCEL SERVI...","{'id': 125, 'name': 'To Be Paid'}","00002X2807193 - DD: 5/22, PD: 5/26",183.58,183.58,2023-05-17T15:43:59.28,2023-05-17T15:43:59.28,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,11400,11255,2023-07-01T00:00:00,"{'id': 3007, 'name': 'Oxford Health Insurance,...","{'id': 125, 'name': 'To Be Paid'}",035133805477 - JUNE 2023 - AP: 6/5 - ED: 6/2,14819.81,14819.81,2023-05-17T15:16:42.327,2023-05-17T15:16:56.63,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,11399,11254,2023-06-14T00:00:00,"{'id': 1458, 'name': 'AMBASSADOR MEDICAL SERVI...","{'id': 125, 'name': 'To Be Paid'}",599464; PD: 6/9,60.00,60.00,2023-05-17T14:37:40.807,2023-05-17T14:37:40.807,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,11398,11253,2023-05-16T00:00:00,"{'id': 32, 'name': 'CHASE CARD SERVICES'}","{'id': 125, 'name': 'To Be Paid'}",3/27/23 - 4/26/23; DD: 06/20 PD: 05/19,12893.80,12893.80,2023-05-17T13:36:24.373,2023-05-17T13:57:11.67,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
bills_single_cols = ["vendor", "status", "currency"]
bills_nested_cols = []
bills_drop_cols = []
bills_date_cols = ["dueDate", "dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
bills_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/bills.csv", 
           bills_single_cols, bills_nested_cols, bills_drop_cols, bills_date_cols)

bills_clean.head()

,id,number,dueDate,memo,totalAmount,openBalance,dateCreated,lastUpdatedDate,vendor_id,vendor_name,status_id,status_name,currency_currencyISOCode,currency_exchangeRate
0,11402,11257,2023-06-16,I76630,415.45,415.45,2023-05-18,2023-05-18,192,STREET CRANE COMPANY LTD,125,To Be Paid,USD,1.0
1,11401,11256,2023-05-23,"00002X2807193 - DD: 5/22, PD: 5/26",183.58,183.58,2023-05-17,2023-05-17,213,UPS - UNITED PARCEL SERVICE,125,To Be Paid,USD,1.0
2,11400,11255,2023-07-01,035133805477 - JUNE 2023 - AP: 6/5 - ED: 6/2,14819.81,14819.81,2023-05-17,2023-05-17,3007,"Oxford Health Insurance, Inc",125,To Be Paid,USD,1.0
3,11399,11254,2023-06-14,599464; PD: 6/9,60.00,60.00,2023-05-17,2023-05-17,1458,AMBASSADOR MEDICAL SERVICES,125,To Be Paid,USD,1.0
4,11398,11253,2023-05-16,3/27/23 - 4/26/23; DD: 06/20 PD: 05/19,12893.80,12893.80,2023-05-17,2023-05-17,32,CHASE CARD SERVICES,125,To Be Paid,USD,1.0


###Collecting and cleaning the vendors Table:

In [ ]:
# call the function with an access token to retrieve all vendors
vendors = get_all_resources(access_token, 'vendors')
vendors.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/vendors.csv", index=False)
vendors.head()

,id,name,number,status,fullAddress,phoneNumber,phoneExtension,phoneNumberTypeId,paymentTerms,dateCreated,lastUpdatedDate
0,2376,ABC NJ ATF,2376,"{'id': 2, 'name': 'Active'}",C/O Associated Builders & Contractors- New Jer...,None,None,0,None,2019-12-09T14:53:19.58,2019-12-09T14:53:47.717
1,3,ABILITY & DRIVE,3,"{'id': 2, 'name': 'Active'}","105 WEST DEWEY AVE, BLDG B, UNIT #8, Wharton, ...",None,None,0,None,2018-01-18T08:17:28.727,2018-01-18T08:17:28.727
2,4,ACCO MATERIAL HANDLING SOLUTIONS,4,"{'id': 2, 'name': 'Active'}","DEPT. CH 16736, PALATINE, IL 60055-6736, USA",None,None,0,None,2018-01-18T08:17:29.737,2020-12-09T13:43:20.027
3,1762,Accuform,1762,"{'id': 1, 'name': 'Prospect'}","16228 Flight Path Drive, Brooksville, FL 34604...",8002371001,,2,None,2018-08-24T12:04:22.87,2018-08-24T12:04:23.707
4,5,ACE OFFICE SOLUTIONS,5,"{'id': 3, 'name': 'Deleted'}",None,9087040400,,2,None,2018-01-18T08:17:29.963,2021-07-27T09:59:32.557


In [ ]:
# Define variables for cleaning data
vendors_single_cols = ["status"]
vendors_nested_cols = []
vendors_drop_cols = []
vendors_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
vendors_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/vendors.csv", 
           vendors_single_cols, vendors_nested_cols, vendors_drop_cols, vendors_date_cols)

vendors_clean.head()

,id,name,number,fullAddress,phoneNumber,phoneExtension,phoneNumberTypeId,paymentTerms,dateCreated,lastUpdatedDate,status_id,status_name
0,2376,ABC NJ ATF,2376,C/O Associated Builders & Contractors- New Jer...,NaN,NaN,0,NaN,2019-12-09,2019-12-09,2,Active
1,3,ABILITY & DRIVE,3,"105 WEST DEWEY AVE, BLDG B, UNIT #8, Wharton, ...",NaN,NaN,0,NaN,2018-01-18,2018-01-18,2,Active
2,4,ACCO MATERIAL HANDLING SOLUTIONS,4,"DEPT. CH 16736, PALATINE, IL 60055-6736, USA",NaN,NaN,0,NaN,2018-01-18,2020-12-09,2,Active
3,1762,Accuform,1762,"16228 Flight Path Drive, Brooksville, FL 34604...",8.002371e+09,NaN,2,NaN,2018-08-24,2018-08-24,1,Prospect
4,5,ACE OFFICE SOLUTIONS,5,NaN,9.087040e+09,NaN,2,NaN,2018-01-18,2021-07-27,3,Deleted


###Collecting and Cleaning the Tasks Table:

In [ ]:
# call the function with an access token to retrieve all tasks
tasks = get_all_resources(access_token, 'tasks')
tasks.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/tasks.csv", index=False)
tasks.head()

,id,taskName,accountId,accountName,assignedTo,dateCreated,lastUpdatedDate,status,statusId,attachmentCount,attachmentLastUploaded
0,24,Chester Hoist Test Bed Design,478,"CONSOLIDATED NUCLEAR SECURITY, LLC - CNS",Al Mehr,2023-02-01T12:47:42.393,2023-03-20T10:38:46.437,Done,50,0,None
1,23,New Task,1,Shupper-Brickle Equipment,Alison Kelley,2022-01-14T16:38:31.57,2022-01-14T16:38:32.107,Open,48,0,None
2,22,aDDING A TASK,1,Shupper-Brickle Equipment,Amy Litecky,2022-01-14T16:35:51.877,2022-01-14T16:40:53.317,Done,50,0,None
3,21,JANUARY 2019 - WEEK #1 - 1/3/19 THRU 1/8/19,802,JLL-MERCK (RAHWAY SITE),Tammy McGinness,2019-06-26T11:23:33.127,2019-06-26T11:23:44.443,Open,48,0,None
4,20,Sabine River Shop Drawings,566,DOW SRO - SABINE RIVER (TX),Brian Trethaway,2019-03-28T16:18:24.12,2019-03-28T16:18:52.523,Open,48,0,None


###Collecting and Cleaning the vendor_locations Table:


In [ ]:
# call the function with an access token to retrieve all vendor_locations
vendor_locations = get_all_resources(access_token, 'vendor-locations')
vendor_locations.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/vendor_locations.csv", index=False)
vendor_locations.head()

,phone,id,name,vendor,isPrimary,address
0,None,3689,"I.D. SYSTEMS, INC.","{'id': 2567, 'number': '2567', 'name': 'I.D. S...",True,{'address1': 'BANK OF AMERICAN LOCKBOX SERVICE...
1,None,277,AMERICAN CRANE & HOIST,"{'id': 282, 'number': '282', 'name': 'AMERICAN...",True,"{'address1': '1234 WASHINGTON STREET ', 'addre..."
2,None,30,CERTIFIED STEEL COMPANY,"{'id': 31, 'number': '31', 'name': 'CERTIFIED ...",True,"{'address1': '1333 BRUNSWICK PIKE', 'address2'..."
3,None,3747,RICCIARDI BROTHERS,"{'id': 175, 'number': '175', 'name': 'RICCIARD...",False,"{'address1': '975 NEW DURHAM RD ', 'address2':..."
4,None,3567,SHARP ELECTRONICS CORPORATION,"{'id': 2499, 'number': '2499', 'name': 'SHARP ...",True,"{'address1': '500 ROSS STREET 154-0455 ', 'add..."


In [ ]:
# Define variables for cleaning data
vendor_locations_single_cols = ["vendor", "address"]
vendor_locations_nested_cols = []
vendor_locations_drop_cols = []
vendor_locations_date_cols = []

# Call clean_data function for each dataset
vendor_locations_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/vendor_locations.csv", 
           vendor_locations_single_cols, vendor_locations_nested_cols, vendor_locations_drop_cols, vendor_locations_date_cols)

vendor_locations_clean.head()

,phone,id,name,isPrimary,vendor_id,vendor_number,vendor_name,address_address1,address_address2,address_address3,address_city,address_state,address_postalCode,address_country,address_latitude,address_longitude,address_fullAddress
0,NaN,3689,"I.D. SYSTEMS, INC.",True,2567,2567,"I.D. SYSTEMS, INC.",BANK OF AMERICAN LOCKBOX SERVICES,62264 COLLECTIONS CENTER DRIVE,,CHICAGO,IL,60693,US,0.0,0.0,"BANK OF AMERICAN LOCKBOX SERVICES , 62264 COLL..."
1,NaN,277,AMERICAN CRANE & HOIST,True,282,282,AMERICAN CRANE & HOIST,1234 WASHINGTON STREET,,,BOSTON,MA,02118,US,0.0,0.0,"1234 WASHINGTON STREET , BOSTON , MA 02118, USA"
2,NaN,30,CERTIFIED STEEL COMPANY,True,31,31,CERTIFIED STEEL COMPANY,1333 BRUNSWICK PIKE,SUITE 200,,LAWRENCEVILLE,NJ,08648,US,0.0,0.0,"1333 BRUNSWICK PIKE, SUITE 200, LAWRENCEVILLE,..."
3,NaN,3747,RICCIARDI BROTHERS,False,175,175,RICCIARDI BROTHERS,975 NEW DURHAM RD,,,EDISON,NJ,08817,US,0.0,0.0,"975 NEW DURHAM RD , EDISON , NJ 08817, USA"
4,NaN,3567,SHARP ELECTRONICS CORPORATION,True,2499,2499,SHARP ELECTRONICS CORPORATION,500 ROSS STREET 154-0455,ATTN: PO BOX 358194,,PITTSBURGH,PA,15262-0001,US,0.0,0.0,"500 ROSS STREET 154-0455 , ATTN: PO BOX 358194..."


###Collecting and Cleaning the bill_credits Table

In [ ]:
# call the function with an access token to retrieve all bill_credits
bill_credits = get_all_resources(access_token, 'bill-credits')
bill_credits.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/bill_credits.csv", index=False)
bill_credits.head()

,id,number,dueDate,vendor,status,memo,totalAmount,openBalance,dateCreated,lastUpdatedDate,currency
0,11184,145,None,"{'id': 201, 'name': 'THERN'}","{'id': 165, 'name': 'Active'}",MI-151789CM,1106.62,0.0,2023-04-06T15:10:42.847,2023-04-06T15:10:50.283,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,11101,144,None,"{'id': 35, 'name': 'CM INDUSTRIAL'}","{'id': 165, 'name': 'Active'}",INV1907227/RG#212317; CM ORDER #5630733/ PO#4956,707.00,0.0,2023-03-24T12:02:39.457,2023-04-21T13:46:01.91,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,11010,143,None,"{'id': 35, 'name': 'CM INDUSTRIAL'}","{'id': 165, 'name': 'Active'}",CREDIT FOR CM HOIST WARRANTY INV# 1906878,2229.68,0.0,2023-03-01T16:32:14.9,2023-04-21T13:46:33.983,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,10989,142,None,"{'id': 40, 'name': 'COOPER ELECTRIC SUPPLY CO.'}","{'id': 165, 'name': 'Active'}",CREDIT FOR BILL RELATED TO PO#5397,38.43,0.0,2023-02-28T09:19:34.66,2023-04-28T14:29:29.357,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,10785,141,None,"{'id': 4, 'name': 'ACCO MATERIAL HANDLING SOLU...","{'id': 165, 'name': 'Active'}",CREDIT FOR FREIGHT CHARGES INVOICE #1503982,23.60,0.0,2023-01-20T13:32:19.797,2023-01-20T13:32:26.96,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
bill_credits_single_cols = ["vendor", "currency", "status"]
bill_credits_nested_cols = []
bill_credits_drop_cols = []
bill_credits_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
bill_credits_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/bill_credits.csv", 
           bill_credits_single_cols, bill_credits_nested_cols, bill_credits_drop_cols, bill_credits_date_cols)

bill_credits_clean.head()

,id,number,dueDate,memo,totalAmount,openBalance,dateCreated,lastUpdatedDate,vendor_id,vendor_name,currency_currencyISOCode,currency_exchangeRate,status_id,status_name
0,11184,145,NaN,MI-151789CM,1106.62,0.0,2023-04-06,2023-04-06,201,THERN,USD,1.0,165,Active
1,11101,144,NaN,INV1907227/RG#212317; CM ORDER #5630733/ PO#4956,707.00,0.0,2023-03-24,2023-04-21,35,CM INDUSTRIAL,USD,1.0,165,Active
2,11010,143,NaN,CREDIT FOR CM HOIST WARRANTY INV# 1906878,2229.68,0.0,2023-03-01,2023-04-21,35,CM INDUSTRIAL,USD,1.0,165,Active
3,10989,142,NaN,CREDIT FOR BILL RELATED TO PO#5397,38.43,0.0,2023-02-28,2023-04-28,40,COOPER ELECTRIC SUPPLY CO.,USD,1.0,165,Active
4,10785,141,NaN,CREDIT FOR FREIGHT CHARGES INVOICE #1503982,23.60,0.0,2023-01-20,2023-01-20,4,ACCO MATERIAL HANDLING SOLUTIONS,USD,1.0,165,Active


###Collecting and Cleaning the sales_orders Table

In [ ]:
# call the function with an access token to retrieve all sales_orders
sales_orders = get_all_resources(access_token, 'sales-orders')
sales_orders.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders.csv", index=False)
sales_orders.head()

,id,number,name,customer,status,dateCreated,lastUpdatedDate
0,12488,201085,Thern Winch,"{'id': 3512, 'name': 'Gloucester City'}","{'id': 19, 'name': 'Quoted'}",2023-05-18T15:11:15.447,2023-05-18T15:12:22.59
1,12487,201084,Thern Davit Crane,"{'id': 1673, 'name': 'Municipal Maintenance Co...","{'id': 19, 'name': 'Quoted'}",2023-05-18T14:32:21.353,2023-05-18T14:32:21.353
2,12486,201083,PARTS; PO R-23-043-3; CM LODESTAR PARTS,"{'id': 3511, 'name': 'USA Crane & Hoist Servic...","{'id': 27, 'name': 'Completed'}",2023-05-18T14:12:53.263,2023-05-18T16:32:56.06
3,12485,201082,SERVICE; PO MRK01633626 - ANNUAL OSHA EQUIPMEN...,"{'id': 1985, 'name': 'CBRE/Merck (WEST POINT ...","{'id': 25, 'name': 'In Progress'}",2023-05-18T12:38:56.997,2023-05-18T12:41:48.897
4,12484,201081,SERVICE; PO 4500161193/137058; RAHWAY (MAXIMO)...,"{'id': 2319, 'name': 'SODEXO INC'}","{'id': 25, 'name': 'In Progress'}",2023-05-18T12:30:46.407,2023-05-18T12:35:40.77


In [ ]:
# Define variables for cleaning data
sales_orders_single_cols = ["customer", "status"]
sales_orders_nested_cols = []
sales_orders_drop_cols = []
sales_orders_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
sales_orders_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders.csv", 
           sales_orders_single_cols, sales_orders_nested_cols, sales_orders_drop_cols, sales_orders_date_cols)

sales_orders_clean.head()

,id,number,name,dateCreated,lastUpdatedDate,customer_id,customer_name,status_id,status_name
0,12488,201085,Thern Winch,2023-05-18,2023-05-18,3512,Gloucester City,19,Quoted
1,12487,201084,Thern Davit Crane,2023-05-18,2023-05-18,1673,Municipal Maintenance Company,19,Quoted
2,12486,201083,PARTS; PO R-23-043-3; CM LODESTAR PARTS,2023-05-18,2023-05-18,3511,USA Crane & Hoist Services,27,Completed
3,12485,201082,SERVICE; PO MRK01633626 - ANNUAL OSHA EQUIPMEN...,2023-05-18,2023-05-18,1985,CBRE/Merck (WEST POINT SITE),25,In Progress
4,12484,201081,SERVICE; PO 4500161193/137058; RAHWAY (MAXIMO)...,2023-05-18,2023-05-18,2319,SODEXO INC,25,In Progress


###Collecting and Cleaning the purchase_orders Table:



In [ ]:
# call the function with an access token to retrieve all purchase_orders
purchase_orders = get_all_resources(access_token, 'purchase-orders')
purchase_orders.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/purchase_orders.csv", index=False)
purchase_orders.head()

,id,poNumber,poName,vendorId,vendor,poTotal,dateCreated,lastUpdatedDate,currency
0,5396,5846,None,0,"{'id': 42, 'name': 'COPPA'S SERVICE CENTER'}",0.00,2023-05-18T13:34:56.62,2023-05-18T13:57:40.403,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,5395,5845,None,0,"{'id': 201, 'name': 'THERN'}",3114.30,2023-05-18T10:46:05.853,2023-05-18T11:08:07.593,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,5394,5844,None,0,"{'id': 4, 'name': 'ACCO MATERIAL HANDLING SOLU...",28250.00,2023-05-18T10:02:22.527,2023-05-18T10:54:04.45,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,5393,5843,None,0,"{'id': 55, 'name': 'DUCT-O-WIRE'}",989.73,2023-05-17T14:05:15.267,2023-05-17T14:10:07.31,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,5392,5842,None,0,"{'id': 192, 'name': 'STREET CRANE COMPANY LTD'}",395.45,2023-05-17T13:54:17.96,2023-05-18T08:23:15.25,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
purchase_orders_single_cols = ["vendor", "currency"]
purchase_orders_nested_cols = []
purchase_orders_drop_cols = []
purchase_orders_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
purchase_orders_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/purchase_orders.csv", 
           purchase_orders_single_cols, purchase_orders_nested_cols, purchase_orders_drop_cols, purchase_orders_date_cols)

purchase_orders_clean.head()

,id,poNumber,poName,vendorId,poTotal,dateCreated,lastUpdatedDate,vendor_id,vendor_name,currency_currencyISOCode,currency_exchangeRate
0,5396,5846,NaN,0,0.00,2023-05-18,2023-05-18,42,COPPA'S SERVICE CENTER,USD,1.0
1,5395,5845,NaN,0,3114.30,2023-05-18,2023-05-18,201,THERN,USD,1.0
2,5394,5844,NaN,0,28250.00,2023-05-18,2023-05-18,4,ACCO MATERIAL HANDLING SOLUTIONS,USD,1.0
3,5393,5843,NaN,0,989.73,2023-05-17,2023-05-17,55,DUCT-O-WIRE,USD,1.0
4,5392,5842,NaN,0,395.45,2023-05-17,2023-05-18,192,STREET CRANE COMPANY LTD,USD,1.0


###Collecting and Cleaning the projects Table



In [ ]:
# call the function with an access token to retrieve all projects
projects = get_all_resources(access_token, 'projects')
projects.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/projects.csv", index=False)
projects.head()

,id,name,customer,type,status,dateCreated,createdBy,lastUpdatedDate,lastUpdatedBy
0,1,181213 - FLUOR,"{'id': 664, 'number': '664', 'name': 'FLUOR EN...","{'id': 2, 'name': 'Bridge Crane'}","{'id': 37, 'name': 'Done'}",2018-09-28T15:28:38.01,"{'id': 22, 'name': 'Brian Trethaway'}",2019-03-28T16:22:30.197,"{'id': 22, 'name': 'Brian Trethaway'}"
1,2,2019 OSHA INSPECTIONS,"{'id': 802, 'number': '802', 'name': 'JLL-MERC...",None,"{'id': 34, 'name': 'Pending'}",2019-06-26T11:20:21.66,"{'id': 17, 'name': 'Tammy Giancone'}",2019-06-26T11:20:21.66,"{'id': 17, 'name': 'Tammy Giancone'}"
2,3,2019 OSHA INSPECTIONS,None,"{'id': 1, 'name': 'Standard Project'}","{'id': 34, 'name': 'Pending'}",2019-06-26T11:20:38.52,"{'id': 17, 'name': 'Tammy Giancone'}",2019-06-26T11:20:38.52,"{'id': 17, 'name': 'Tammy Giancone'}"
3,4,2019 OSHA INSPECTIONS,"{'id': 802, 'number': '802', 'name': 'JLL-MERC...","{'id': 1, 'name': 'Standard Project'}","{'id': 36, 'name': 'In Progress'}",2019-06-26T11:21:02.957,"{'id': 17, 'name': 'Tammy Giancone'}",2019-06-26T11:22:04.897,"{'id': 17, 'name': 'Tammy Giancone'}"
4,5,2019 OSHA INSPECTIONS,"{'id': 802, 'number': '802', 'name': 'JLL-MERC...","{'id': 1, 'name': 'Standard Project'}","{'id': 36, 'name': 'In Progress'}",2019-06-26T11:41:46.073,"{'id': 17, 'name': 'Tammy Giancone'}",2019-06-26T11:44:33.44,"{'id': 17, 'name': 'Tammy Giancone'}"


In [ ]:
# Define variables for cleaning data
projects_single_cols = ["customer", "type", "status", "createdBy", "lastUpdatedBy"]
projects_nested_cols = []
projects_drop_cols = []
projects_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
projects_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/projects.csv", 
           projects_single_cols, projects_nested_cols, projects_drop_cols, projects_date_cols)

projects_clean.head()

,id,name,dateCreated,lastUpdatedDate,customer_id,customer_number,customer_name,type_id,type_name,status_id,status_name,createdBy_id,createdBy_name,lastUpdatedBy_id,lastUpdatedBy_name
0,1,181213 - FLUOR,2018-09-28,2019-03-28,664.0,664,FLUOR ENTERPRISES INC.,2.0,Bridge Crane,37,Done,22,Brian Trethaway,22,Brian Trethaway
1,2,2019 OSHA INSPECTIONS,2019-06-26,2019-06-26,802.0,802,JLL-MERCK (RAHWAY SITE),NaN,NaN,34,Pending,17,Tammy Giancone,17,Tammy Giancone
2,3,2019 OSHA INSPECTIONS,2019-06-26,2019-06-26,NaN,NaN,NaN,1.0,Standard Project,34,Pending,17,Tammy Giancone,17,Tammy Giancone
3,4,2019 OSHA INSPECTIONS,2019-06-26,2019-06-26,802.0,802,JLL-MERCK (RAHWAY SITE),1.0,Standard Project,36,In Progress,17,Tammy Giancone,17,Tammy Giancone
4,5,2019 OSHA INSPECTIONS,2019-06-26,2019-06-26,802.0,802,JLL-MERCK (RAHWAY SITE),1.0,Standard Project,36,In Progress,17,Tammy Giancone,17,Tammy Giancone


###Collecting and Cleaning the payments Table



In [ ]:
# call the function with an access token to retrieve all payments
payments = get_all_resources(access_token, 'payments')
payments.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/payments.csv", index=False)
payments.head()

,id,referenceNumber,paymentDate,customer,status,paymentAmount,openBalance,dateCreated,lastUpdatedDate,currency
0,7400,AMEX/9040,2023-05-18T00:00:00,"{'id': 3511, 'name': 'USA Crane & Hoist Servic...","{'id': 70, 'name': 'Approved'}",155.81,0.0,2023-05-18T16:34:35.847,2023-05-18T16:34:36.507,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,7399,246570,2023-05-18T00:00:00,"{'id': 384, 'name': 'BREHOB CRANE AND HOIST'}","{'id': 70, 'name': 'Approved'}",527.63,0.0,2023-05-18T12:34:13.073,2023-05-18T12:34:13.393,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,7398,900055,2023-05-18T00:00:00,"{'id': 2887, 'name': 'SMITH & SONS'}","{'id': 70, 'name': 'Approved'}",1540.04,0.0,2023-05-18T12:30:09.243,2023-05-18T12:30:18.07,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,7397,AMEX/9000,2023-05-18T00:00:00,"{'id': 928, 'name': 'Mid Atlantic Mechanical'}","{'id': 70, 'name': 'Approved'}",553.71,0.0,2023-05-18T08:34:34.69,2023-05-18T08:34:35.09,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,7396,ACH,2023-05-18T00:00:00,"{'id': 1125, 'name': 'RADWELL INTERNATIONAL IN...","{'id': 70, 'name': 'Approved'}",2288.63,0.0,2023-05-18T08:18:02.137,2023-05-18T08:18:02.68,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
payments_single_cols = ["customer", "status", "currency"]
payments_nested_cols = []
payments_drop_cols = []
payments_date_cols = ["paymentDate", "dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
payments_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/payments.csv", 
           payments_single_cols, payments_nested_cols, payments_drop_cols, payments_date_cols)

payments_clean.head()

,id,referenceNumber,paymentDate,paymentAmount,openBalance,dateCreated,lastUpdatedDate,customer_id,customer_name,status_id,status_name,currency_currencyISOCode,currency_exchangeRate
0,7400,AMEX/9040,2023-05-18,155.81,0.0,2023-05-18,2023-05-18,3511,USA Crane & Hoist Services,70,Approved,USD,1.0
1,7399,246570,2023-05-18,527.63,0.0,2023-05-18,2023-05-18,384,BREHOB CRANE AND HOIST,70,Approved,USD,1.0
2,7398,900055,2023-05-18,1540.04,0.0,2023-05-18,2023-05-18,2887,SMITH & SONS,70,Approved,USD,1.0
3,7397,AMEX/9000,2023-05-18,553.71,0.0,2023-05-18,2023-05-18,928,Mid Atlantic Mechanical,70,Approved,USD,1.0
4,7396,ACH,2023-05-18,2288.63,0.0,2023-05-18,2023-05-18,1125,RADWELL INTERNATIONAL INC.,70,Approved,USD,1.0


###Collecting and Cleaning the opportunities Table

In [ ]:
# call the function with an access token to retrieve all opportunities
opportunities = get_all_resources(access_token, 'opportunities')
opportunities.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/opportunities.csv", index=False)
opportunities.head()

,id,title,stage,opportunityType,customer,dateCreated,lastUpdatedDate
0,1,,"{'id': 1, 'name': 'Open'}","{'id': 1, 'name': 'Sales Opportunity'}","{'id': 1534, 'name': '#1534 - CHAZ ZYGMONT'}",2018-02-09T10:08:51.077,2018-02-09T10:08:51.077
1,2,,"{'id': 1, 'name': 'Open'}","{'id': 1, 'name': 'Sales Opportunity'}","{'id': 1623, 'name': '#1623 - DANISCO - CANADA...",2018-04-26T14:57:23.323,2018-04-26T14:57:23.323


In [ ]:
# Define variables for cleaning data
opportunities_single_cols = ["stage", "opportunityType", "customer"]
opportunities_nested_cols = []
opportunities_drop_cols = []
opportunities_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
opportunities_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/opportunities.csv", 
           opportunities_single_cols, opportunities_nested_cols, opportunities_drop_cols, opportunities_date_cols)

opportunities_clean.head()

,id,title,dateCreated,lastUpdatedDate,stage_id,stage_name,opportunityType_id,opportunityType_name,customer_id,customer_name
0,1,NaN,2018-02-09,2018-02-09,1,Open,1,Sales Opportunity,1534,#1534 - CHAZ ZYGMONT
1,2,NaN,2018-04-26,2018-04-26,1,Open,1,Sales Opportunity,1623,#1623 - DANISCO - CANADA INC


###Collecting and Cleaning the items Table

In [ ]:
# call the function with an access token to retrieve all items
items = get_all_resources(access_token, 'items')
items.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/items.csv", index=False)
items.head()

,id,name,itemNumber,itemType,description,price,cost,taxable,isShippingRequired,shippingPrice,shippingPerc,glCategory,inventoryLocation,maxQuantity,active,dateCreated,lastUpdatedDate
0,2462,"#LEGEND, LEGEND SHEET FOR PENDANTS",None,"{'id': 1, 'name': 'Inventory'}","#LEGEND, LEGEND SHEET FOR PENDANTS",2.00,1.14,True,False,0.0,0.0,None,"{'id': 1, 'name': 'Main SB Warehouse'}",0.0,True,2019-09-12T11:58:47.83,2022-02-01T10:09:23.96
1,430,**INACTIVATED --LHH674 6 CAPACITY LABEL,None,"{'id': 1, 'name': 'Inventory'}",None,4.20,3.57,True,False,0.0,0.0,None,"{'id': 1, 'name': 'Main SB Warehouse'}",NaN,False,2018-01-30T13:23:04.607,2018-09-11T11:23:14.153
2,1164,00000171 INT SHAFT S/A 92T/8T,None,"{'id': 1, 'name': 'Inventory'}",None,309.04,216.33,True,False,0.0,0.0,None,"{'id': 1, 'name': 'Main SB Warehouse'}",NaN,True,2018-01-30T13:24:02.173,2018-07-02T09:45:01.22
3,1349,00000224B CONTACT BLOCK,None,"{'id': 1, 'name': 'Inventory'}",None,52.31,36.62,True,False,0.0,0.0,None,"{'id': 1, 'name': 'Main SB Warehouse'}",NaN,True,2018-01-30T13:24:16.663,2018-07-02T09:44:54.647
4,2571,00000236 GASKET,None,"{'id': 1, 'name': 'Inventory'}",GASKET,8.55,5.99,True,False,0.0,0.0,None,"{'id': 1, 'name': 'Main SB Warehouse'}",0.0,True,2020-03-24T09:07:30.013,2020-05-06T13:38:11.387


In [ ]:
# Define variables for cleaning data
items_single_cols = ["itemType", "inventoryLocation"]
items_nested_cols = []
items_drop_cols = []
items_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
items_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/items.csv", 
           items_single_cols, items_nested_cols, items_drop_cols, items_date_cols)

items_clean.head()

,id,name,itemNumber,description,price,cost,taxable,isShippingRequired,shippingPrice,shippingPerc,glCategory,maxQuantity,active,dateCreated,lastUpdatedDate,itemType_id,itemType_name,inventoryLocation_id,inventoryLocation_name
0,2462,"#LEGEND, LEGEND SHEET FOR PENDANTS",NaN,"#LEGEND, LEGEND SHEET FOR PENDANTS",2.00,1.14,True,False,0.0,0.0,NaN,0.0,True,2019-09-12,2022-02-01,1,Inventory,1.0,Main SB Warehouse
1,430,**INACTIVATED --LHH674 6 CAPACITY LABEL,NaN,NaN,4.20,3.57,True,False,0.0,0.0,NaN,NaN,False,2018-01-30,2018-09-11,1,Inventory,1.0,Main SB Warehouse
2,1164,00000171 INT SHAFT S/A 92T/8T,NaN,NaN,309.04,216.33,True,False,0.0,0.0,NaN,NaN,True,2018-01-30,2018-07-02,1,Inventory,1.0,Main SB Warehouse
3,1349,00000224B CONTACT BLOCK,NaN,NaN,52.31,36.62,True,False,0.0,0.0,NaN,NaN,True,2018-01-30,2018-07-02,1,Inventory,1.0,Main SB Warehouse
4,2571,00000236 GASKET,NaN,GASKET,8.55,5.99,True,False,0.0,0.0,NaN,0.0,True,2020-03-24,2020-05-06,1,Inventory,1.0,Main SB Warehouse


###Collecting and Cleaning the item_builds Table

In [ ]:
# call the function with an access token to retrieve all item_builds
item_builds = get_all_resources(access_token, 'item-builds')
item_builds.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/item_builds.csv", index=False)
item_builds.head()

,id,refNumber,item,status,buildDate,dateCreated,lastUpdatedDate
0,2,2,"{'id': 1207, 'name': '36800Y CONTROL STATION ...","{'id': 170, 'name': 'Finalized'}",2018-08-31T00:00:00,2018-08-31T11:11:47.92,None
1,1,1,"{'id': 2071, 'name': '26600-ASM CONTROL STATIO...","{'id': 170, 'name': 'Finalized'}",2018-07-30T00:00:00,2018-07-30T10:57:35.583,None


In [ ]:
# Define variables for cleaning data
item_builds_single_cols = ["item", "status"]
item_builds_nested_cols = []
item_builds_drop_cols = []
item_builds_date_cols = ["dateCreated", "buildDate"]

# Call clean_data function for each dataset
item_builds_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/item_builds.csv", 
           item_builds_single_cols, item_builds_nested_cols, item_builds_drop_cols, item_builds_date_cols)

item_builds_clean.head()

,id,refNumber,buildDate,dateCreated,lastUpdatedDate,item_id,item_name,status_id,status_name
0,2,2,2018-08-31,2018-08-31,NaN,1207,36800Y CONTROL STATION (YELLOW) 2 SPEED,170,Finalized
1,1,1,2018-07-30,2018-07-30,NaN,2071,"26600-ASM CONTROL STATION, CONDUCTIX",170,Finalized


###Collecting and Cleaning customers the Table

In [ ]:
# call the function with an access token to retrieve all customers
customers = get_all_resources(access_token, 'customers')
customers.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/customers.csv", index=False)
customers.head()

,id,name,number,status,customerSince,fullAddress,countryDialCode,phoneNumber,phoneExtension,phoneNumberTypeId,paymentTerms,dateCreated,lastUpdatedDate
0,226,None,226,"{'id': 3, 'name': 'Deleted'}",None,None,0,None,None,0,None,2018-01-18T08:19:40.107,2018-01-18T08:28:04.207
1,3114,#3113 - VERTEX LLC,3114,"{'id': 1, 'name': 'Prospect'}",None,"2330 SCENIC HWY, SUITE 223, SNELLVILLE, GA 300...",0,None,None,0,None,2022-04-25T13:03:03.7,2022-04-25T14:40:43.097
2,1733,???????????????,1733,"{'id': 3, 'name': 'Deleted'}",None,None,0,None,None,0,None,2018-08-07T09:32:14.823,2018-08-31T12:35:16.58
3,227,"174th Air National Guard, Hancock Field",227,"{'id': 2, 'name': 'Active'}",None,"6001 E. Molloy Road, CE BLDG 645, Syracuse, NY...",0,None,None,0,None,2018-01-18T08:33:23.92,2018-08-27T10:30:22.46
4,3343,3 DOTS SERVICES INC,3343,"{'id': 1, 'name': 'Prospect'}",None,"12626 Kingston Springs Ct, Houston, TX 77089, USA",0,None,None,0,None,2022-11-30T11:34:22.537,2022-11-30T12:03:23.17


In [ ]:
# Define variables for cleaning data
customers_single_cols = ["status"]
customers_nested_cols = []
customers_drop_cols = []
customers_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
customers_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/customers.csv", 
           customers_single_cols, customers_nested_cols, customers_drop_cols, customers_date_cols)

customers_clean.head()

,id,name,number,customerSince,fullAddress,countryDialCode,phoneNumber,phoneExtension,phoneNumberTypeId,paymentTerms,dateCreated,lastUpdatedDate,status_id,status_name
0,226,NaN,226,NaN,NaN,0,NaN,NaN,0,NaN,2018-01-18,2018-01-18,3,Deleted
1,3114,#3113 - VERTEX LLC,3114,NaN,"2330 SCENIC HWY, SUITE 223, SNELLVILLE, GA 300...",0,NaN,NaN,0,NaN,2022-04-25,2022-04-25,1,Prospect
2,1733,???????????????,1733,NaN,NaN,0,NaN,NaN,0,NaN,2018-08-07,2018-08-31,3,Deleted
3,227,"174th Air National Guard, Hancock Field",227,NaN,"6001 E. Molloy Road, CE BLDG 645, Syracuse, NY...",0,NaN,NaN,0,NaN,2018-01-18,2018-08-27,2,Active
4,3343,3 DOTS SERVICES INC,3343,NaN,"12626 Kingston Springs Ct, Houston, TX 77089, USA",0,NaN,NaN,0,NaN,2022-11-30,2022-11-30,1,Prospect


###Collecting and Cleaning the customer-locations Table

In [ ]:
# call the function with an access token to retrieve all customer_locations
customer_locations = get_all_resources(access_token, 'customer-locations')
customer_locations.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/customer_locations.csv", index=False)
customer_locations.head()

,phones,id,name,customer,isPrimary,address
0,None,2626,\t FLEMINGTON GRANITE,"{'id': 496, 'number': '496', 'name': 'CRETER V...",False,"{'address1': '417 US HIGHWAY 202', 'address2':..."
1,None,2319,\t NEW JERSEY AMERICAN WATER,"{'id': 999, 'number': '999', 'name': 'NJ AMERI...",True,"{'address1': '1 WATER STREET', 'address2': 'AP..."
2,None,277,AMERICAN CRANE & HOIST,"{'id': 282, 'number': '282', 'name': 'AMERICAN...",True,"{'address1': '1234 WASHINGTON STREET ', 'addre..."
3,None,3356,APM Terminals,"{'id': 2375, 'number': '2375', 'name': 'APM Te...",True,"{'address1': '5080 McLester Street', 'address2..."
4,None,3238,Atlantic Steel Solutions LLC,"{'id': 2305, 'number': '2305', 'name': 'Atlant...",True,"{'address1': '74 Railroad Avenue', 'address2':..."


In [ ]:
# Define variables for cleaning data
customer_locations_single_cols = ["address", "customer"]
customer_locations_nested_cols = []
customer_locations_drop_cols = []
customer_locations_date_cols = []

# Call clean_data function for each dataset
customer_locations_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/customer_locations.csv", 
           customer_locations_single_cols, customer_locations_nested_cols, customer_locations_drop_cols, customer_locations_date_cols)

customer_locations_clean.head()

,phones,id,name,isPrimary,address_address1,address_address2,address_address3,address_city,address_state,address_postalCode,address_country,address_latitude,address_longitude,address_fullAddress,customer_id,customer_number,customer_name
0,NaN,2626,\t FLEMINGTON GRANITE,False,417 US HIGHWAY 202,,,FLEMINGTON,NJ,08822,US,0.000000,0.000000,"417 US HIGHWAY 202, FLEMINGTON , NJ 08822, USA",496,496,CRETER VAULT CORP
1,NaN,2319,\t NEW JERSEY AMERICAN WATER,True,1 WATER STREET,APT 1018,,CAMDEN,NJ,08102-1658,US,0.000000,0.000000,"1 WATER STREET, APT 1018, CAMDEN, NJ 08102-165...",999,999,NJ AMERICAN WATER
2,NaN,277,AMERICAN CRANE & HOIST,True,1234 WASHINGTON STREET,,,BOSTON,MA,02118,US,0.000000,0.000000,"1234 WASHINGTON STREET , BOSTON , MA 02118, USA",282,282,AMERICAN CRANE & HOIST
3,NaN,3356,APM Terminals,True,5080 McLester Street,,,Elizabeth,NJ,07201,US,40.669467,-74.159026,"5080 McLester Street, Elizabeth, NJ 07201, USA",2375,2375,APM Terminals
4,NaN,3238,Atlantic Steel Solutions LLC,True,74 Railroad Avenue,,,Paterson,NJ,07501,US,40.912679,-74.168014,"74 Railroad Avenue, Paterson, NJ 07501, USA",2305,2305,Atlantic Steel Solutions LLC


###Collecting and Cleaning the customer_assets Table

In [ ]:
# call the function with an access token to retrieve all customer_assets
customer_assets = get_all_resources(access_token, 'customer-assets')
customer_assets.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/customer_assets.csv", index=False)
customer_assets.head()

""


###Collecting and Cleaning the credit_memos Table

In [ ]:
# call the function with an access token to retrieve all credit_memos
credit_memos = get_all_resources(access_token, 'credit-memos')
credit_memos.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/credit_memos.csv", index=False)
credit_memos.head()

,id,txnNumber,customer,creditMemoTotal,openBalance,memo,dateCreated,lastUpdatedDate,currency
0,8531,CM1907440,"{'id': 964, 'name': 'MOTION INDUSTRIES GROUP'}",1.49,0.0,Credit for: Invoice 1907440,2023-05-09T11:21:41.61,2023-05-09T11:21:41.61,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
1,8524,CM1907434,"{'id': 3499, 'name': 'Armstrong Engineering'}",1.94,0.0,Credit for: Invoice 1907434,2023-05-05T13:30:35.137,2023-05-05T13:30:35.137,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
2,8429,CM1907340,"{'id': 3473, 'name': 'Carol Woods'}",0.96,0.0,Credit for: Invoice 1907340,2023-04-14T12:31:38.503,2023-04-14T12:31:38.503,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
3,8412,CM1907116,"{'id': 3171, 'name': 'Reading Crane and Engine...",4032.62,0.0,Credit for: Invoice 1907116 RETURNED PARTS,2023-04-12T09:52:01.05,2023-04-12T09:52:01.05,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"
4,8387,CM1907300,"{'id': 3458, 'name': 'Methuen Construction'}",4.61,0.0,Credit for: Invoice 1907300,2023-04-06T15:23:19.963,2023-04-06T15:23:19.963,"{'currencyISOCode': 'USD', 'exchangeRate': 1.0}"


In [ ]:
# Define variables for cleaning data
credit_memos_single_cols = ["customer", "currency"]
credit_memos_nested_cols = []
credit_memos_drop_cols = []
credit_memos_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
credit_memos_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/credit_memos.csv", 
           credit_memos_single_cols, credit_memos_nested_cols, credit_memos_drop_cols, credit_memos_date_cols)

credit_memos_clean.head()

,id,txnNumber,creditMemoTotal,openBalance,memo,dateCreated,lastUpdatedDate,customer_id,customer_name,currency_currencyISOCode,currency_exchangeRate
0,8531,CM1907440,1.49,0.0,Credit for: Invoice 1907440,2023-05-09,2023-05-09,964,MOTION INDUSTRIES GROUP,USD,1.0
1,8524,CM1907434,1.94,0.0,Credit for: Invoice 1907434,2023-05-05,2023-05-05,3499,Armstrong Engineering,USD,1.0
2,8429,CM1907340,0.96,0.0,Credit for: Invoice 1907340,2023-04-14,2023-04-14,3473,Carol Woods,USD,1.0
3,8412,CM1907116,4032.62,0.0,Credit for: Invoice 1907116 RETURNED PARTS,2023-04-12,2023-04-12,3171,Reading Crane and Engineering Company,USD,1.0
4,8387,CM1907300,4.61,0.0,Credit for: Invoice 1907300,2023-04-06,2023-04-06,3458,Methuen Construction,USD,1.0


###Collecting and Cleaning the contacts Table

In [ ]:
# call the function with an access token to retrieve all contacts
contacts = get_all_resources(access_token, 'contacts')
contacts.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/contacts.csv", index=False)
contacts.head()

,id,firstName,lastName,dateCreated,lastUpdatedDate,active
0,10666,LINCOLN,CHIN,2023-05-18T15:04:55.54,2023-05-18T15:04:55.593,True
1,10665,KENNETH,PATRICCO,2023-05-18T15:04:10.21,2023-05-18T15:04:10.25,True
2,10664,Ray,Bennett,2023-05-18T15:02:03.937,2023-05-18T15:02:04.247,True
3,10663,Chris,Hellings,2023-05-18T14:11:35.233,2023-05-18T14:11:35.44,True
4,10662,KEVIN,RADUE,2023-05-18T14:07:27.867,2023-05-18T14:07:28.327,True


In [ ]:
# Define variables for cleaning data
contacts_single_cols = []
contacts_nested_cols = []
contacts_drop_cols = []
contacts_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
contacts_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/contacts.csv", 
           contacts_single_cols, contacts_nested_cols, contacts_drop_cols, contacts_date_cols)

contacts_clean.head()

,id,firstName,lastName,dateCreated,lastUpdatedDate,active
0,10666,LINCOLN,CHIN,2023-05-18,2023-05-18,True
1,10665,KENNETH,PATRICCO,2023-05-18,2023-05-18,True
2,10664,Ray,Bennett,2023-05-18,2023-05-18,True
3,10663,Chris,Hellings,2023-05-18,2023-05-18,True
4,10662,KEVIN,RADUE,2023-05-18,2023-05-18,True


###Collecting and Cleaning the appointments Table

In [ ]:
# call the function with an access token to retrieve all appointments
appointments = get_all_resources(access_token, 'appointments')
appointments.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/appointments_N.csv", index=False)
appointments.head()

""


###Collecting and Cleaning the sales_orders_details Table

In [ ]:
#get the first row of sales_orders_details

#id =sales_orders["id"][0]
#detail = get_details(id, access_token, "https://api.striven.com/v1/sales_orders/")
#detail.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details.csv") 

In [ ]:
# API endpoint and file paths
api_url="https://api.striven.com/v1/sales-orders/"
path_table = "/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders.csv"
path_detail = "/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details.csv"

# Set this flag to True to start updating the sales order details

while pd.read_csv(path_table).shape[0] > pd.read_csv(path_detail).shape[0]:
    # Update the sales order details and print the shape of the resulting DataFrame
    try:
        details = update_details(
            access_token,
            api_url=api_url,
            path_table=path_table,
            path_detail=path_detail,
            x=10)
        print("Updated details shape:", details.shape)
    except Exception as e:
        print(e)  # Print the exception message
        break

<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (12452, 40)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (12462, 40)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (12472, 40)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (12482, 40)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (12487, 40)


In [ ]:
'''
# Define variables for cleaning data
sales_orders_details_single_cols = ["contact", "type", "customer", "billToLocation", "shipToLocation", "salesRep", "paymentTerm", "arglAccount", "invoiceFormat", "salesTax", "status", "invoiceStatus", "createdBy", "currency", "lastUpdatedBy"]
sales_orders_details_nested_cols = ["lineItems", "customFields"]
sales_orders_details_drop_cols = ["orderDate", "dateCreated", "lastUpdatedDate"]
sales_orders_details_date_cols = ["shipVia", "trackingNumber", "project", "lineItemsClass", "lineItems_unitOfMeasure", "lineItems_fulfillmentDate", "lineItems_class", "lineItems_itemGroupLineItems", "customFields_valueText"]

# Call clean_data function for each dataset
sales_orders_details_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details.csv", 
           sales_orders_details_single_cols, sales_orders_details_nested_cols, sales_orders_details_drop_cols, sales_orders_details_date_cols)

sales_orders_details_clean.head()

'''
#sales_orders_details_clean[sales_orders_details_clean["type_name"]=="Parts"]

'\n# Define variables for cleaning data\nsales_orders_details_single_cols = ["contact", "type", "customer", "billToLocation", "shipToLocation", "salesRep", "paymentTerm", "arglAccount", "invoiceFormat", "salesTax", "status", "invoiceStatus", "createdBy", "currency", "lastUpdatedBy"]\nsales_orders_details_nested_cols = ["lineItems", "customFields"]\nsales_orders_details_drop_cols = ["orderDate", "dateCreated", "lastUpdatedDate"]\nsales_orders_details_date_cols = ["shipVia", "trackingNumber", "project", "lineItemsClass", "lineItems_unitOfMeasure", "lineItems_fulfillmentDate", "lineItems_class", "lineItems_itemGroupLineItems", "customFields_valueText"]\n\n# Call clean_data function for each dataset\nsales_orders_details_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details.csv", \n           sales_orders_details_single_cols, sales_orders_details_nested_cols, sales_orders_details_drop_cols, sales_orders_details_date_cols)\n\nsales_orders_details_clean.he

In [ ]:
sales_orders_details_clean = pd.read_csv("/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details.csv")
sales_orders_details_clean = sales_orders_details_clean.applymap(try_literal_eval)
sales_orders_details_clean = sales_orders_details_clean[sales_orders_details_clean["lineItems"].notnull()]

single_columns = ["contact", "type", "customer", "billToLocation", "shipToLocation", "salesRep", "paymentTerm", "arglAccount", "invoiceFormat", "salesTax", "status", "invoiceStatus", "createdBy", "currency", "lastUpdatedBy"]

nested_columns = ["lineItems", "customFields"]

date_columns = ["orderDate", "dateCreated", "lastUpdatedDate"]

drop_columns = ["shipVia", "trackingNumber", "project", "lineItemsClass", "lineItems_unitOfMeasure", "lineItems_fulfillmentDate", "lineItems_class", "lineItems_itemGroupLineItems", "customFields_valueText"]

for col in single_columns:
    sales_orders_details_clean = add_clean_col(sales_orders_details_clean, col)

for col in nested_columns:
    sales_orders_details_clean = add_clean_col(sales_orders_details_clean, col, nested=True)

for col in date_columns:
    sales_orders_details_clean[col] = pd.to_datetime(sales_orders_details_clean[col], errors='coerce').dt.date

#sales_orders_details_clean = sales_orders_details_clean.drop(drop_columns, axis=1)

sales_orders_details_clean["type_name"] = sales_orders_details_clean["type_name"].str.title()

#sales_orders_details_clean = sales_orders_details_clean[["id", "orderNumber", "orderName", "createdBy_name", "status_name", "type_name", "dateCreated", "orderTotal"]]

sales_orders_details_clean.to_csv('/content/drive/My Drive/Projects/Shupper_Brickle/sales_orders_details_clean.csv', index=False)
sales_orders_details_clean.sample(3)



,id,shipVia,trackingNumber,project,orderDate,orderNumber,orderName,customerPONumber,toBePrinted,targetDate,customerNotes,internalNotes,lineItemsClass,orderTotal,isChangeOrder,parentId,isRecurring,notesLogCount,attachmentCount,dateCreated,lastUpdatedDate,type_id,type_name,contact_id,contact_name,customer_id,customer_number,customer_name,billToLocation_id,billToLocation_name,shipToLocation_id,shipToLocation_name,salesRep_id,salesRep_name,paymentTerm_id,paymentTerm_name,arglAccount_id,arglAccount_name,invoiceFormat_id,invoiceFormat_name,salesTax_id,salesTax_name,status_id,status_name,invoiceStatus_id,invoiceStatus_name,createdBy_id,createdBy_name,currency_currencyISOCode,currency_exchangeRate,lastUpdatedBy_id,lastUpdatedBy_name,lineItems_id,lineItems_item,lineItems_inventoryLocation,lineItems_description,lineItems_qty,lineItems_unitOfMeasure,lineItems_price,lineItems_shippingPrice,lineItems_taxable,lineItems_fulfillmentDate,lineItems_class,lineItems_itemGroupLineItems,lineItems_customColumns,customFields_id,customFields_name,customFields_fieldType,customFields_sourceId,customFields_value,customFields_valueText,customFields_isRequired
9885,10822,NaN,NaN,NaN,2022-08-18,199356,PARTS; PO 2600882788 - ELECTROLIFT SHEAVES,2600882788,False,2022-08-30T00:00:00,"{'id': 28017, 'notesHtml': '<p>&nbsp;</p><p><s...","{'id': 28092, 'notesHtml': 'INVD: 8/23/22', 'n...",NaN,481.44,False,0,False,0,3,2022-08-18,2022-08-23,1958,Parts,NaN,NaN,567,567,DUPONT - SPRUANCE (VA),2913.0,"P.O. BOX 80040\r\n WILMINGTON, DE, 19880-0040...",1638.0,"5200 DUPONT SITE ROAD\r\nRICHMOND, VA, 23234",17.0,Tammy Giancone,3,Net 30,2,1101 - 1101 Accounts Receivable,50,*Invoice-Main-No Model#s*,16.0,Exempt,27,Completed,169.0,Full,17,Tammy Giancone,USD,1.0,17.0,Tammy Giancone,35931,"{'id': 1976, 'name': 'PARTS (NI)'}",None,"ELECTROLIFT \nSHEAVE, STEEL, PART# K130SCP1",1.0,None,481.44,0.0,True,None,None,None,[],37,Proposal Reference,"{'id': 3, 'name': 'Alphanumeric Field'}",0,ELECTROLIFT SHEAVES - PART NO. PROVIDED BY CUS...,None,False
6263,6265,NaN,NaN,NaN,2020-06-30,194802,EQUIPMENT; CATTRON RADIO CONTROL SYSTEM,NaN,False,2020-07-30T00:00:00,"{'id': 16969, 'notesHtml': '<div><span id=""txt...",NaN,NaN,1840.41,False,0,False,0,2,2020-06-30,2020-06-30,1957,Equipment,2109.0,Michael Kinsey,500,500,CTK INDUSTRIAL SERVICE - DO NOT DO BUSINESS - ...,495.0,"919 SPRUCE STREET\r\nMontoursville, PA, 17754",495.0,"919 SPRUCE STREET\r\nMontoursville, PA, 17754",7.0,Daniel Cahill,3,Net 30,2,1101 - 1101 Accounts Receivable,28,*Invoice-Main-No Model#s*,16.0,Exempt,19,Quoted,167.0,No,17,Tammy Giancone,USD,1.0,17.0,Tammy Giancone,18863,"{'id': 1975, 'name': 'EQUIPMENT'}",None,RADIO CONTROL SYSTEM\nITEM# 325 10R-AC F\n\nFR...,1.0,None,1170.00,0.0,True,None,None,None,[],37,Proposal Reference,"{'id': 3, 'name': 'Alphanumeric Field'}",0,CATTRON RADIO CONTROL SYSTEM,None,False
4459,4461,NaN,NaN,NaN,2019-10-01,193000,PARTS; PO 45845 - 8-BUTTON SBP2 ENCLOSURES,45845,False,2019-06-07T00:00:00,"{'id': 12668, 'notesHtml': '<div><span id=""txt...","{'id': 12742, 'notesHtml': None, 'notesText': ...",NaN,945.00,False,0,False,0,0,2019-10-01,2019-10-04,1958,Parts,1096.0,Jim Millward,1353,1353,WASTE GAS,1341.0,"450 NEW BOLD ROAD\r\nFAIRLESS HILLS , PA, 1903...",1341.0,"450 NEW BOLD ROAD\r\nFAIRLESS HILLS , PA, 1903...",7.0,Daniel Cahill,3,Net 30,2,1101 - 1101 Accounts Receivable,17,OLDInvoice-Main-ShowingModel#s,16.0,Exempt,27,Completed,169.0,Full,17,Tammy Giancone,USD,1.0,33.0,Renee Rivera,12552,"{'id': 704, 'name': 'SBP2-E8 8-BUTTON ENCLOSU...","{'id': 1, 'name': 'Main SB Warehouse'}",MAGNETEK REPLACEMENT ENCLOSURE (PENDANT HOUSIN...,6.0,None,157.50,0.0,True,None,None,None,[],37,Proposal Reference,"{'id': 3, 'name': 'Alphanumeric Field'}",0,MAGNETEK 8-BUTTON SBP2 ENCLOSURES,None,False


###Collecting and Cleaning the invoices_details Table

In [ ]:
#get the first row of invoices_details

#id =invoices["id"][0]
#detail = get_details(id, access_token, "https://api.striven.com/v1/invoices/")
#detail.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/invoices_details.csv") 

In [ ]:
# API endpoint and file paths
api_url="https://api.striven.com/v1/invoices/"
path_table = "/content/drive/My Drive/Projects/Shupper_Brickle/invoices.csv"
path_detail = "/content/drive/My Drive/Projects/Shupper_Brickle/invoices_details.csv"

# Set this flag to True to start updating the sales order details

while pd.read_csv(path_table).shape[0] > pd.read_csv(path_detail).shape[0]:
    # Update the sales order details and print the shape of the resulting DataFrame
    try:
        details = update_details(
            access_token,
            api_url=api_url,
            path_table=path_table,
            path_detail=path_detail,
            x=10)
        print("Updated details shape:", details.shape)
    except Exception as e:
        print(e)  # Print the exception message
        break

<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (8185, 33)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (8195, 33)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (8203, 33)


In [ ]:
# Define variables for cleaning data
invoices_details_single_cols = ["paymentTerms", "customer", "billToLocation", "shipToLocation", "salesRep", "status", "arAccount", "order", "format", "createdBy", "currency", "lineItems_item"]
invoices_details_nested_cols = ["lineItems"]
invoices_details_drop_cols = ["shipVia", "trackingNumber", "lineItems_customColumns", "lineItems_itemGroupLineItems", "lineItems_glCategory", "lineItems_fulfillmentDate", "lineItems_unitOfMeasure", "lineItems_glAccount", "currency_currencyISOCode", "currency_exchangeRate", "arAccount_id", "reviewedBy", "dateReviewed", "reviewed", "sendInvoiceNotification", "doNotChargeConvenienceFee", "glCategory", "allowDuplicateTxnNumber"]
invoices_details_date_cols = ["txnDate", "dueDate", "dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
invoices_details_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/invoices_details.csv", 
           invoices_details_single_cols, invoices_details_nested_cols, invoices_details_drop_cols, invoices_details_date_cols)

invoices_details_clean.head()

,id,txnNumber,txnDate,dueDate,memo,openBalance,toBePrinted,taxId,attachmentCount,dateCreated,lastUpdatedDate,lastUpdatedBy,lineItems_id,lineItems_location,lineItems_description,lineItems_qty,lineItems_price,lineItems_shippingPrice,lineItems_taxable,paymentTerms_id,paymentTerms_name,customer_id,customer_number,customer_name,billToLocation_id,billToLocation_name,shipToLocation_id,shipToLocation_name,salesRep_id,salesRep_name,status_id,status_name,arAccount_name,order_id,order_name,format_id,format_name,createdBy_id,createdBy_name,lineItems_item_id,lineItems_item_name
0,8285,1907207,2023-03-14,2023-06-12,ANNUAL OSHA EQUIPMENT INSPECTION 2023 - WK#5,1296.0,True,16,1,2023-03-17,NaT,NaN,19557,None,3/13/23 - LABOR: 8 HRS @ $81/HR (TECH #1)\n,8.00,81.00,0.00,False,10.0,Net 90,1985,1985,CBRE/Merck (WEST POINT SITE),3417.0,"PO BOX 982122\r\n EL PASO, TX, 79998-2122, USA",2657.0,"770 Sumneytown Pike\r\nWest Point, PA, 19486, USA",7.0,Daniel Cahill,160,Active,1101 - 1101 Accounts Receivable,12057.0,SERVICE; PO MRK01633626 - ANNUAL OSHA EQUIPMEN...,50,*Invoice-Main-No Model#s*,45,Jessica Dyevoich,1977,INSPECTION
1,8275,1907198,2023-03-14,2023-03-14,SHIPPED 3/14/23; TRK#1Z2X28070268533766,0.0,False,16,0,2023-03-14,NaT,NaN,19544,"{'id': 1, 'name': 'Main SB Warehouse'}","BOARD CONV TERM, PART# 28828\nDELIVERY: 1 IN ...",1.00,150.93,27.99,True,5.0,Credit Card,3439,3439,SGPS/ShowRig Boston,5299.0,"92 Blandin Ave, Suite G\r\nFramingham, MA, 017...",5299.0,"92 Blandin Ave, Suite G\r\nFramingham, MA, 017...",17.0,Tammy Giancone,160,Active,1101 - 1101 Accounts Receivable,12093.0,PARTS; PO 1086 - CM PARTS,50,*Invoice-Main-No Model#s*,45,Jessica Dyevoich,1427,28828 BOARD CONV TERM
2,8277,1907199,2023-03-15,2023-03-15,MILESTONE INVOICE - 25% DOWN PAYMENT OF ORDER ...,6270.5,False,3,0,2023-03-15,NaT,NaN,19546,None,RELOCATE 5 TON FREESTANDING CRANE SYSTEM\n\nSC...,0.25,25082.00,0.00,True,1.0,Due On Receipt,1274,1274,ABB INSTALLATION PRODUCTS INC.,2586.0,"1 ESNA DRIVE \r\nHACKETTSTOWN, NJ, 07840, USA",2585.0,"13 ROUTE 57 \r\nHACKETTSTOWN , NJ, 07840, USA",7.0,Daniel Cahill,160,Active,1101 - 1101 Accounts Receivable,11992.0,SERVICE; PO MD1575 - RELOCATE 5 TON FREESTANDI...,50,*Invoice-Main-No Model#s*,45,Jessica Dyevoich,1974,SERVICE
3,8278,1907200,2023-03-15,2023-04-14,SERVICE VISIT - NO TROLLEY MOVEMENT,1065.0,False,16,0,2023-03-15,NaT,NaN,19547,None,"3-15-23 \nT/C PVF2 REACTOR CRANE, NO TROLLEY ...",1.00,1065.00,0.00,True,3.0,Net 30,551,551,DUPONT - FAYETTEVILLE (NC),4420.0,"P.O. BOX 80040\r\nWilmington, DE, 19880-004, USA",1775.0,"22828 NC HIGHWAY 87 WEST \r\n FAYETTEVILLE, NC...",6.0,Glenn Billand,160,Active,1101 - 1101 Accounts Receivable,12090.0,SERVICE; PO 7500035393 - PVF2 T/C,50,*Invoice-Main-No Model#s*,45,Jessica Dyevoich,1974,SERVICE
4,8279,1907201,2023-03-15,2023-04-14,SHIPPED 3/15/23; TRK#1Z2X28070370190371,0.0,False,3,0,2023-03-15,NaT,NaN,19548,None,"29335, PENDANT AND 18 FT CORD ASSEMBLY",1.00,824.00,0.00,True,3.0,Net 30,732,732,HENRIQUES YACHTS,726.0,"198 Hilton Ave\r\nBayville, NJ, 8721",726.0,"198 Hilton Ave\r\nBayville, NJ, 8721",6.0,Glenn Billand,160,Active,1101 - 1101 Accounts Receivable,11930.0,PARTS; PO VERBAL NATALIE - CM LODESTAR PENDANT...,32,OLD-Main Invoice-w/Model#'s,45,Jessica Dyevoich,1976,PARTS (NI)


In [ ]:
invoices_details_clean[invoices_details_clean["txnNumber"]==1907201]

,id,txnNumber,txnDate,dueDate,memo,openBalance,toBePrinted,taxId,attachmentCount,dateCreated,lastUpdatedDate,lastUpdatedBy,lineItems_id,lineItems_location,lineItems_description,lineItems_qty,lineItems_price,lineItems_shippingPrice,lineItems_taxable,paymentTerms_id,paymentTerms_name,customer_id,customer_number,customer_name,billToLocation_id,billToLocation_name,shipToLocation_id,shipToLocation_name,salesRep_id,salesRep_name,status_id,status_name,arAccount_name,order_id,order_name,format_id,format_name,createdBy_id,createdBy_name,lineItems_item_id,lineItems_item_name
4,8279,1907201,2023-03-15,2023-04-14,SHIPPED 3/15/23; TRK#1Z2X28070370190371,0.0,False,3,0,2023-03-15,NaT,NaN,19548,None,"29335, PENDANT AND 18 FT CORD ASSEMBLY",1.0,824.0,0.0,True,3.0,Net 30,732,732,HENRIQUES YACHTS,726.0,"198 Hilton Ave\r\nBayville, NJ, 8721",726.0,"198 Hilton Ave\r\nBayville, NJ, 8721",6.0,Glenn Billand,160,Active,1101 - 1101 Accounts Receivable,11930.0,PARTS; PO VERBAL NATALIE - CM LODESTAR PENDANT...,32,OLD-Main Invoice-w/Model#'s,45,Jessica Dyevoich,1976,PARTS (NI)


###Collecting and Cleaning the bills_details Table

In [ ]:
#get the first row of bills_details

#id =bills["id"][0]
#detail = get_details(id, access_token, "https://api.striven.com/v1/bills/")
#detail.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/bills_details.csv") 

In [ ]:
# API endpoint and file paths
api_url="https://api.striven.com/v1/bills/"
path_table = "/content/drive/My Drive/Projects/Shupper_Brickle/bills.csv"
path_detail = "/content/drive/My Drive/Projects/Shupper_Brickle/bills_details.csv"

# Set this flag to True to start updating the sales order details

while pd.read_csv(path_table).shape[0] > pd.read_csv(path_detail).shape[0]:
    # Update the sales order details and print the shape of the resulting DataFrame
    try:
        details = update_details(
            access_token,
            api_url=api_url,
            path_table=path_table,
            path_detail=path_detail,
            x=10)
        time.sleep(3)
        print("Updated details shape:", details.shape)
    except Exception as e:
        print(e)  # Print the exception message
        break

<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (11244, 30)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pa

Updated details shape: (11254, 30)


<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)
<ipython-input-3-fb6b8baaec5c>:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  details = details.append(data, ignore_index=True)


Updated details shape: (11257, 30)


In [ ]:
# Define variables for cleaning data
bills_details_single_cols = ["paymentTerm", "vendor", "vendorLocation", "shippingMethod", "requestedBy", "apglAccount", "status", "currency", "createdBy", "lastUpdatedBy", "format", "lineItems_item", "lineItems_glAccount", "lineItems_customer", "lineItems_order"]
bills_details_nested_cols = ["lineItems"]
bills_details_drop_cols = ["trackingLink", "lineItemsGLCategory", "dateReviewed", "reviewedBy", "lineItems_unitOfMeasure", "lineItems_glCategory" ]
bills_details_date_cols = ['txnDate', 'dueDate', 'dateCreated', 'lastUpdatedDate']

# Call clean_data function for each dataset
bills_details_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/bills_details.csv", 
           bills_details_single_cols, bills_details_nested_cols, bills_details_drop_cols, bills_details_date_cols)

bills_details_clean.head()

,id,txnNumber,txnDate,dueDate,referenceNumber,memo,billTotal,openBalance,isItemReceipt,notesLogCount,attachmentCount,dateCreated,lastUpdatedDate,reviewed,lineItems_id,lineItems_isExpense,lineItems_qty,lineItems_cost,lineItems_description,lineItems_purchaseOrderLineItemId,lineItems_billable,lineItems_billed,lineItems_inventoryLocation,lineItems_customColumns,paymentTerm_id,paymentTerm_name,vendor_id,vendor_number,vendor_name,vendorLocation_id,vendorLocation_name,shippingMethod_id,shippingMethod_name,requestedBy_id,requestedBy_name,apglAccount_id,apglAccount_name,status_id,status_name,currency_currencyISOCode,currency_exchangeRate,createdBy_id,createdBy_name,lastUpdatedBy_id,lastUpdatedBy_name,format_id,format_name,lineItems_item_id,lineItems_item_name,lineItems_glAccount_id,lineItems_glAccount_name,lineItems_customer_id,lineItems_customer_name,lineItems_order_id,lineItems_order_name
0,11088,00002X2807113,2023-03-18,2023-03-28,00002X2807113,"00002X2807113 - DD: 3/27, PD: 3/31",113.11,113.11,False,0,0,2023-03-22,NaT,False,2447817,False,1.0,17.79,OUTGOING - 3/8/23,0,False,False,None,[],2.0,Net 10,213,213,UPS - UNITED PARCEL SERVICE,210.0,Primary Location,NaN,NaN,NaN,NaN,7,2001 Accounts Payable,125,To Be Paid,USD,1.0,45.0,Jessica Dyevoich,NaN,NaN,51,Bill Format,1979.0,FREIGHT,NaN,None,3427.0,PLANET GRANITE INC,12028.0,#200626 - PARTS; PO 1083 - HARRINGTON PARTS
1,1,1461347,2017-12-22,2018-02-20,1461347,NaN,1747.61,0.00,False,0,0,2018-02-02,2018-02-16,False,55734,False,1.0,1747.61,None,0,False,False,None,[],7.0,Net 60,4,4,ACCO MATERIAL HANDLING SOLUTIONS,1656.0,Primary Location,NaN,NaN,1.0,Brian Kinney (Striven Support),7,2001 Accounts Payable,126,Paid,USD,1.0,1.0,Brian Kinney (Striven Support),9.0,Ellen Frauenheim,51,Bill Format,4.0,Opening Balance Equity Item,NaN,None,NaN,None,NaN,None
2,2,1461371,2017-12-27,2018-02-25,1461371,NaN,773.54,0.00,False,0,0,2018-02-02,2018-02-26,False,55741,False,1.0,773.54,None,0,False,False,None,[],7.0,Net 60,4,4,ACCO MATERIAL HANDLING SOLUTIONS,1656.0,Primary Location,NaN,NaN,1.0,Brian Kinney (Striven Support),7,2001 Accounts Payable,126,Paid,USD,1.0,1.0,Brian Kinney (Striven Support),9.0,Ellen Frauenheim,51,Bill Format,4.0,Opening Balance Equity Item,NaN,None,NaN,None,NaN,None
3,3,1461425,2018-01-02,2018-03-03,1461425,NaN,13120.98,0.00,False,0,0,2018-02-02,2018-03-02,False,55742,False,1.0,13120.98,None,0,False,False,None,[],7.0,Net 60,4,4,ACCO MATERIAL HANDLING SOLUTIONS,1656.0,Primary Location,NaN,NaN,1.0,Brian Kinney (Striven Support),7,2001 Accounts Payable,126,Paid,USD,1.0,1.0,Brian Kinney (Striven Support),9.0,Ellen Frauenheim,51,Bill Format,4.0,Opening Balance Equity Item,NaN,None,NaN,None,NaN,None
4,4,1461575,2018-01-09,2018-03-10,1461575,NaN,1650.46,0.00,False,0,0,2018-02-02,2018-03-09,False,55743,False,1.0,1650.46,,0,False,False,None,[],7.0,Net 60,4,4,ACCO MATERIAL HANDLING SOLUTIONS,1656.0,Primary Location,NaN,NaN,1.0,Brian Kinney (Striven Support),7,2001 Accounts Payable,126,Paid,USD,1.0,1.0,Brian Kinney (Striven Support),9.0,Ellen Frauenheim,51,Bill Format,4.0,Opening Balance Equity Item,NaN,None,NaN,None,NaN,None


###Collecting and Cleaning the itmes_details Table

In [ ]:
#get the first row of items_details

#id =items["id"][0]
#detail = get_details(id, access_token, "https://api.striven.com/v1/items/")
#detail.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/items_details.csv") 

In [ ]:
# API endpoint and file paths
api_url="https://api.striven.com/v1/items/"
path_table = "/content/drive/My Drive/Projects/Shupper_Brickle/items.csv"
path_detail = "/content/drive/My Drive/Projects/Shupper_Brickle/items_details.csv"  

# Set this flag to True to start updating the sales order details

while pd.read_csv(path_table).shape[0] > pd.read_csv(path_detail).shape[0]:
    # Update the sales order details and print the shape of the resulting DataFrame
    try:
        details = update_details(
            access_token,
            api_url=api_url,
            path_table=path_table,
            path_detail=path_detail,
            x=10)
        print("Updated details shape:", details.shape)
    except Exception as e:
        print(e)  # Print the exception message
        break

In [ ]:
# Define variables for cleaning data
items_details_single_cols = ["itemType", "category", "preferredVendor", "manufacturer", "internalNotes", "incomeGLAccount", "inventoryAssetGLAccount", "cogsglAccount", "defaultInventoryLocation", "createdBy", "lastUpdatedBy"]
items_details_nested_cols = ["customFields"]
items_details_drop_cols = ["groupItems", "baseUnitOfMeasure", "barcodeValue", "customFields_valueText", "Unnamed: 0", "division", "defaultGLCategory", "relatedAsset", "baseUnitOfMeasure", "barcodeValue", "customFields_valueText", ]
items_details_date_cols = ["dateCreated", "lastUpdatedDate"]

# Call clean_data function for each dataset
items_details_clean = clean_data("/content/drive/My Drive/Projects/Shupper_Brickle/items_details.csv", 
           items_details_single_cols, items_details_nested_cols, items_details_drop_cols, items_details_date_cols)

items_details_clean.head()

,id,name,itemNumber,manufacturePartNumber,description,price,cost,taxable,weight,isShippingRequired,shippingPrice,shippingPerc,expenseGLAccount,allowVendorToIncludeInBill,reorderPoint,reorderAmount,qtyOnHand,qtyAvailable,maxQuantity,active,dateCreated,lastUpdatedDate,customFields_id,customFields_name,customFields_fieldType,customFields_sourceId,customFields_value,customFields_isRequired,itemType_id,itemType_name,category_id,category_name,preferredVendor_id,preferredVendor_number,preferredVendor_name,manufacturer_id,manufacturer_number,manufacturer_name,internalNotes_id,internalNotes_notesHtml,internalNotes_notesText,incomeGLAccount_id,incomeGLAccount_name,inventoryAssetGLAccount_id,inventoryAssetGLAccount_name,cogsglAccount_id,cogsglAccount_name,defaultInventoryLocation_id,defaultInventoryLocation_name,createdBy_id,createdBy_name,lastUpdatedBy_id,lastUpdatedBy_name
0,2462,"#LEGEND, LEGEND SHEET FOR PENDANTS",NaN,#LEGEND,"#LEGEND, LEGEND SHEET FOR PENDANTS",2.00,1.14,True,0.00,False,0.0,0.0,NaN,False,0.0,0.0,5.0,5.0,0.0,True,2019-09-12,2022-02-01,29,Model Number,"{'id': 3, 'name': 'Alphanumeric Field'}",0,#LEGEND,False,1,Inventory,9.0,Parts,129.0,129,"MAGNETEK, INC.",129.0,129,"MAGNETEK, INC.",12301.0,"{'id': 12301, 'notesHtml': None, 'notesText': ...",9/12/2019,86,4002 SALES - PARTS,48.0,1301 INVENTORY,98.0,4504 COGS - INVENTORY PARTS,1.0,Main SB Warehouse,33,Renee Rivera,2.0,Alison Kelley
1,2571,00000236 GASKET,NaN,00000236,GASKET,8.55,5.99,True,0.03,False,0.0,0.0,NaN,False,0.0,0.0,1.0,1.0,0.0,True,2020-03-24,2020-05-06,29,Model Number,"{'id': 3, 'name': 'Alphanumeric Field'}",0,00000236,False,1,Inventory,NaN,NaN,35.0,35,CM INDUSTRIAL,35.0,35,CM INDUSTRIAL,15632.0,"{'id': 15632, 'notesHtml': None, 'notesText': ...",3/24/2020,86,4002 SALES - PARTS,48.0,1301 INVENTORY,98.0,4504 COGS - INVENTORY PARTS,1.0,Main SB Warehouse,33,Renee Rivera,33.0,Renee Rivera
2,1164,00000171 INT SHAFT S/A 92T/8T,NaN,171,NaN,309.04,216.33,True,4.95,False,0.0,0.0,NaN,False,0.0,0.0,1.0,1.0,NaN,True,2018-01-30,2018-07-02,29,Model Number,"{'id': 3, 'name': 'Alphanumeric Field'}",0,None,False,1,Inventory,24.0,Hoist Parts,35.0,35,CM INDUSTRIAL,35.0,35,CM INDUSTRIAL,1181.0,"{'id': 1181, 'notesHtml': None, 'notesText': '...",10/10/2017,86,4002 SALES - PARTS,48.0,1301 INVENTORY,98.0,4504 COGS - INVENTORY PARTS,1.0,Main SB Warehouse,1,Brian Kinney (Striven Support),1.0,Brian Kinney (Striven Support)
3,1349,00000224B CONTACT BLOCK,NaN,00000224B,NaN,52.31,36.62,True,1.03,False,0.0,0.0,NaN,False,0.0,0.0,1.0,1.0,NaN,True,2018-01-30,2018-07-02,29,Model Number,"{'id': 3, 'name': 'Alphanumeric Field'}",0,None,False,1,Inventory,9.0,Parts,35.0,35,CM INDUSTRIAL,35.0,35,CM INDUSTRIAL,1366.0,"{'id': 1366, 'notesHtml': None, 'notesText': '...",1/28/2015,86,4002 SALES - PARTS,48.0,1301 INVENTORY,98.0,4504 COGS - INVENTORY PARTS,1.0,Main SB Warehouse,1,Brian Kinney (Striven Support),1.0,Brian Kinney (Striven Support)
4,430,**INACTIVATED --LHH674 6 CAPACITY LABEL,NaN,LHH674 6,NaN,4.20,3.57,True,1.00,False,0.0,0.0,NaN,False,0.0,0.0,0.0,0.0,NaN,False,2018-01-30,2018-09-11,29,Model Number,"{'id': 3, 'name': 'Alphanumeric Field'}",0,LHH674 6,False,1,Inventory,9.0,Parts,35.0,35,CM INDUSTRIAL,36.0,36,COFFING HOISTS,447.0,"{'id': 447, 'notesHtml': None, 'notesText': '5...",5/16/2013,86,4002 SALES - PARTS,48.0,1301 INVENTORY,98.0,4504 COGS - INVENTORY PARTS,1.0,Main SB Warehouse,1,Brian Kinney (Striven Support),2.0,Alison Kelley


###Collecting and Cleaning the Tables:



In [ ]:
# Define variables for each table
referral_sources = get_resources(access_token, 'referral-sources')
item_types = get_resources(access_token, 'item-types')
sales_order_types = get_resources(access_token, 'sales-order-types')
purchase_order_types = get_resources(access_token, 'purchase-order-types')
payment_terms = get_resources(access_token, 'payment-terms')
payment_processing_methods = get_resources(access_token, 'payment-processing-methods')
payment_methods = get_resources(access_token, 'payment-methods')
inventory_locations = get_resources(access_token, 'inventory-locations')
industries = get_resources(access_token, 'industries')
custom_lists = get_resources(access_token, 'custom-lists')
company_locations = get_resources(access_token, 'company-locations')

# Create a list of tables and a corresponding list of API paths
tables = [referral_sources, item_types, sales_order_types, purchase_order_types, payment_terms, payment_processing_methods, payment_methods, inventory_locations, industries, custom_lists, company_locations]
paths = ['referral-sources', 'item-types', 'sales-order-types', 'purchase-order-types', 'payment-terms', 'payment-processing-methods', 'payment-methods', 'inventory-locations', 'industries', 'custom-lists', 'company-locations']

# Loop through the tables and API paths to save each table to a CSV file
for table, path in zip(tables, paths):
    file_path = f"/content/drive/My Drive/Projects/Shupper_Brickle/{path}.csv"
    table.to_csv(file_path, index=False)


###Collecting and Cleaning the gl_accounts Table

In [ ]:
# call the function with an access token to retrieve all gl_accounts
gl_accounts = get_resources_2(access_token, 'gl-accounts')
gl_accounts.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/gl_accounts_N.csv", index=False)
gl_accounts.head()

,id,accountName,accountExtendedName,accountType,parent,accountNumber,doNotAllowPosting,isReconcilable,active
0,43,1000 Assets,1000 - 1000 Assets,"{'id': 11, 'name': 'Other Asset'}",None,1000,True,False,False
1,44,1013 PNC BANK,1013 - 1013 PNC BANK,"{'id': 11, 'name': 'Other Asset'}",None,1013,False,True,False
2,144,1013 PNC BANK,1013 - 1013 PNC BANK,"{'id': 4, 'name': 'Bank'}",None,1013,False,True,True
3,162,1014 FIRST COMMERCE BK 1235,1014 - 1014 FIRST COMMERCE BK 1235,"{'id': 4, 'name': 'Bank'}",None,1014,False,True,True
4,145,1015 FIRST COMMERCE BANK,1015 - 1015 FIRST COMMERCE BANK,"{'id': 4, 'name': 'Bank'}",None,1015,False,True,True


In [ ]:
#gl_accounts[gl_accounts["accountName"]=="4002 - 4002 SALES - PARTS"]
#gl_accounts

In [ ]:
gl_accounts["accountType_id"] = gl_accounts["accountType"].apply(lambda x: x["id"])
gl_accounts["accountType_name"] = gl_accounts["accountType"].apply(lambda x: x["name"])
gl_accounts = gl_accounts.drop("accountType", axis=1)
gl_accounts.head()

,id,accountName,accountExtendedName,parent,accountNumber,doNotAllowPosting,isReconcilable,active,accountType_id,accountType_name
0,43,1000 Assets,1000 - 1000 Assets,None,1000,True,False,False,11,Other Asset
1,44,1013 PNC BANK,1013 - 1013 PNC BANK,None,1013,False,True,False,11,Other Asset
2,144,1013 PNC BANK,1013 - 1013 PNC BANK,None,1013,False,True,True,4,Bank
3,162,1014 FIRST COMMERCE BK 1235,1014 - 1014 FIRST COMMERCE BK 1235,None,1014,False,True,True,4,Bank
4,145,1015 FIRST COMMERCE BANK,1015 - 1015 FIRST COMMERCE BANK,None,1015,False,True,True,4,Bank


###Collecting and Cleaning the pools Table

In [ ]:
def get_pools(access_token):
    """
    Retrieves pools from an API endpoint and returns them as a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the pools API endpoint with the headers.
    response = requests.get("https://api.striven.com/v1/pools", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the pools from the response object.

        pools = response_obj

        # Convert the pools to a Pandas DataFrame.
        df = pd.DataFrame(pools)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

# Call the function with an access token to retrieve all pools.
pools = get_pools(access_token)
pools.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/pools.csv", index=False)
pools.head()

,id,name,isDefault
0,1,Default Pool,True
1,2,Inspection Pool,False
2,3,Manufacturing Pool,False


###Collecting and Cleaning the employees Table

In [ ]:
def get_employees(access_token):
    """
    Retrieves employees from an API endpoint and returns them as a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the employees API endpoint with the headers.
    response = requests.get("https://api.striven.com/v1/employees", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the employees from the response object.

        employees = response_obj

        # Convert the employees to a Pandas DataFrame.
        df = pd.DataFrame(employees)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

# Call the function with an access token to retrieve all employees.
employees = get_employees(access_token)
employees.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/employees.csv", index=False)
employees.head(1000)

,id,name,email,isSystemUser,dateCreated,lastUpdatedDate
0,2,Alison Kelley,akelley@sbomh.com,True,2017-12-04T10:27:27.4,2022-01-27T15:08:49.483
1,3,Timothy Beckett,None,False,2018-01-11T14:37:07.603,2020-06-16T16:06:38.74
2,6,Glenn Billand,glennb@sbomh.com,True,2018-01-11T14:37:08.94,2022-01-27T15:04:59.077
3,7,Daniel Cahill,dcahill@sbomh.com,True,2018-01-11T14:37:09.13,2021-11-30T09:37:18.103
4,10,Edward Giglio,None,False,2018-01-11T14:37:10.32,2021-11-30T09:43:07.8
5,12,"Stephen Hart, Sr.",None,False,2018-01-11T14:37:10.94,2019-07-29T09:28:18.667
6,13,"Stephen Hart, Jr.",None,False,2018-01-11T14:37:11.257,2019-10-01T12:18:12.847
7,14,Amy Litecky,amyl@sbomh.com,True,2018-01-11T14:37:11.61,2022-01-27T15:06:18.183
8,15,Andrew T. Litecky,atl@sbomh.com,True,2018-01-11T14:37:12.25,2023-01-18T14:34:28.49
9,16,Josephine Marques,jmarques@sbomh.com,True,2018-01-11T14:37:12.577,2023-03-27T11:08:18.01


###Collecting and Cleaning the company Table

In [ ]:
def get_company(access_token):
    """
    Retrieves company from an API endpoint and returns them as a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the company API endpoint with the headers.
    response = requests.get("https://api.striven.com/v1/company/profile", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the company from the response object.
        company = response_obj
        
        # Convert the company to a Pandas DataFrame.
        df = pd.DataFrame(company, index=[0])

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

# Call the function with an access token to retrieve all company.
company = get_company(access_token)
company.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/company.csv", index=False)
company.head

<bound method NDFrame.head of                  companyName           email               website  \
0  Shupper-Brickle Equipment  INFO@SBOMH.COM  http://www.sbomh.com   

                  subdomain  utcOffset               timeZone  dateFormat  
0  ShupperBrickleEquipment2  -05:00:00  Eastern Standard Time  MM/dd/yyyy  >

###Collecting and Cleaning the shipping-methods Table:

In [ ]:
def get_shipping_methods(access_token):
    """
    Retrieves shipping_methods from an API endpoint and returns them as a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the shipping_methods API endpoint with the headers.
    response = requests.get("https://api.striven.com/v1/shipping-methods", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the shipping_methods from the response object.

        shipping_methods = response_obj

        # Convert the shipping_methods to a Pandas DataFrame.
        df = pd.DataFrame(shipping_methods)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

# Call the function with an access token to retrieve all shipping_methods.
shipping_methods = get_shipping_methods(access_token)
shipping_methods.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/shipping_methods.csv", index=False)
shipping_methods.head()

,shippingMethodId,shippingMethod,trackingURL,active
0,18,Allowed,None,True
1,19,Collect,None,True
2,21,"DAP, Destination, PPA",None,True
3,22,Fedex Freight,,True
4,23,Fedex Freight Collect,,True


###Collecting and Cleaning the invoice-formats Table

In [ ]:
def get_invoice_formats(access_token):
    """
    Retrieves invoice_formats from an API endpoint and returns them as a Pandas DataFrame.
    """
    # Set the headers for the API call.
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"}

    # Make a GET request to the invoice_formats API endpoint with the headers.
    response = requests.get("https://api.striven.com/v1/invoice-formats", headers=headers)

    # Check the response status code.
    if response.status_code == 200:
        # Deserialize the response JSON string into a Python object.
        response_obj = response.json()
        # Get the invoice_formats from the response object.

        invoice_formats = response_obj

        # Convert the invoice_formats to a Pandas DataFrame.
        df = pd.DataFrame(invoice_formats)

        return df
    else:
        # Return the response text directly if the status code is not 200.
        return response.text

# Call the function with an access token to retrieve all invoice_formats.
invoice_formats = get_invoice_formats(access_token)
invoice_formats.to_csv("/content/drive/My Drive/Projects/Shupper_Brickle/invoice_formats.csv", index=False)
invoice_formats.head()

,id,name,active
0,28,*Invoice-Main-No Model#s*,False
1,50,*Invoice-Main-No Model#s*,True
2,29,*Invoice-PAID Stamp-NoModel#s,True
3,27,*Repair Invoice,False
4,30,Invoice - Commercial Invoice--NoModel#s,True
